In [833]:
import pandas as pd
import pypyodbc as py
import xlsxwriter
import gspread
import gspread_dataframe as gd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g 
from pprint import pprint
from googleapiclient import discovery
import pypyodbc 
import numpy as np 
import sqlalchemy 
import urllib 
import time 
import datetime as dt
from datetime import datetime
import pymssql
import calendar
from google.oauth2 import service_account
import pygsheets
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pygsheets
import json
from google.oauth2 import service_account

In [1118]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.linear_model import LassoCV,RidgeCV,ElasticNetCV
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV,KFold,RandomizedSearchCV,StratifiedKFold,cross_val_score
from sklearn.metrics import r2_score
sns.set_context("paper", font_scale = 1, rc={"grid.linewidth": 3})
pd.set_option('display.max_rows', 100, 'display.max_columns', 400)
from scipy.stats import skew,boxcox_normmax
from scipy.special import boxcox1p
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
import statsmodels.api as sm

In [834]:
# Connect to DB
db = py.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=103.69.193.246;"
                        "Database=dwh;"
                        "uid=dwh;pwd=F88!23456789")
print('Connected to SQL server: DWH')

Connected to SQL server: DWH


In [835]:
with open(r'D:\F88\Python code\Immediate_warning\service_account.json') as source:
    info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

client = pygsheets.authorize(service_account_file=r'D:\F88\Python code\Immediate_warning\service_account.json')

shop_id='1OCzHxTE7Er8_W1-IT92rTOCISAlufFOjcwtw2eIX0tU'
shop_wb=client.open_by_key(shop_id)
shop_detail_new=shop_wb.worksheet_by_title('Everything').get_as_df()

In [836]:
# QLKV, miền
shop_id_mkt='1ZTQE_pfBCAUr-0GSMPJBqtCyZVxX4hdiPuT_RDrdjeQ'
shop_wb_mkt=client.open_by_key(shop_id_mkt)
shop_detail_mkt=shop_wb_mkt.worksheet_by_title('shop_detail').get_as_df()

In [837]:
shop_detail_mkt = shop_detail_mkt[['Mã PGD', 'QLKV']]

In [1158]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'D:\F88\Python code\Immediate_warning\service_account.json', scope)
gc = gspread.authorize(credentials)

client = pygsheets.authorize(service_account_file=r'D:\F88\Python code\Immediate_warning\service_account.json')

#Lấy ID và truy cập wb Backup
backup_ss_id='1iYODUg3bS0JlRzkuq-cnrVZYXJgcNGAa2Wj7i2McHIc'
backup_wb=gc.open_by_key(backup_ss_id)

#Lấy ID và truy cập wb Warning.Control
control_ssheet_id= '1-XjnJNeEwEuk13_xQjmewvZ6aAXFRXSlcEcQpaOq3vE' 
control_wb=gc.open_by_key(control_ssheet_id)

#Function chuyển sheet thành Data Frame
def to_dataframe(wb_name,sheet_name,data_row,col):
    a=wb_name.worksheet(sheet_name).get_all_values()
    return pd.DataFrame(data=a[data_row:],columns=a[col])

#Funtion update data vào sheet wb_name:tên wb,target_sheet: tên sheet cần update; data:dữ liệu muốn update
def update_data(wb_name,target_sheet,data):
    existing =to_dataframe(wb_name,target_sheet,1,0)
    updated = existing.append(data,sort=False)
    gd.set_with_dataframe(wb_name.worksheet(target_sheet),updated)

# Function ghi đè dữ liệu lên gg sheet
def write_data(wb_name,ws_name,ws_range,data):
    wb_name.values_clear(ws_range)
    sheet=wb_name.worksheet(ws_name)
    set_with_dataframe(sheet,data)
    print('Đã ghi dữ liệu lên sheet '+ ws_name)

#Function backup data các HĐ đã check
def back_up_data(data):
    update_data(backup_wb,'backup',data[['codeno','reason_check','shopcode','year_month']])
    short_list=to_dataframe(control_wb,'short_list',1,0)

In [839]:
# Lấy ID shopdetail
shop_detail_id='1ZTQE_pfBCAUr-0GSMPJBqtCyZVxX4hdiPuT_RDrdjeQ'
shop_detail_wb=gc.open_by_key(shop_detail_id)
shop_qlkv_region=to_dataframe(shop_detail_wb,'shop_detail',1,0)

In [771]:
shop_qlkv_region =shop_qlkv_region[['Mã PGD','Miền','QLKV']]

In [772]:
check_date= dt.date.today()+dt.timedelta(days=-2)

check_date = check_date.strftime('%Y-%m-%d')

In [773]:
dic_date = {'key':1, 'extract_date':[check_date]}

In [774]:
date_table = pd.DataFrame.from_dict(dic_date)

In [1210]:
shop_infor = pd.read_sql_query("""Declare @check_date date = '"""+check_date+"""'
select  sd2.code shopcode, sd2.name shopname, sd2.OpenDate
from (select distinct(sd.shopcode) shop_code
from (SELECT case when code = 'SGN0120' then 'SGN0147'
when code = 'SG2046' then 'SGN0126'
when code = 'SGN0107' then 'SG2014'
else code end shopcode
  FROM [dwh].[dbo].[ShopDetail]
  where code not in ('TEST','HS', 'F88LUX','HNLTK','HNIXLN01','SGNXLN01','HNBM1','HOISO')) sd
  left join (select shopcode, max(fromdate) max_cre
	from [dwh].[dbo].pawn group by shopcode) pa on pa.shopcode = sd.shopcode
	where pa.max_cre >= dateadd(d, -2, @check_date)) main
left join [dwh].[dbo].[ShopDetail] sd2 on main.shop_code = sd2.code""", db)

In [1211]:
shop_infor = shop_infor.merge(shop_detail_new[['Mã PGD','Tỉnh/TP', 'Vĩ độ', 'Kinh độ', 'Vùng/Miền']], left_on = 'shopcode', right_on = 'Mã PGD', how = 'left')

In [1212]:
# province_list_id
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'D:\F88\Python code\Immediate_warning\service_account.json', scope)
gc = gspread.authorize(credentials)
fault_group_id='1X7m2UnrVh909OKk9YOkxcIB455GPwTpAAcWGHIBn6_c'
fault_group_wb=gc.open_by_key(fault_group_id)
province_id = to_dataframe(fault_group_wb, 'province_list',1,0)

In [1213]:
shop_infor['key'] = 1

shop_infor = shop_infor.merge(date_table, on = 'key', how = 'left')

In [1214]:
shop_infor['yearmonth'] = [''.join(x.split('-')[0:2]) for x in shop_infor['extract_date']]

In [1215]:
shop_infor = shop_infor.merge(province_id, left_on = ['yearmonth','Tỉnh/TP'], right_on = ['yearmonth','province_name'], how = 'left')

In [1216]:
# shop_age_month = shop_age_month.merge(staff_tenure, on= ['thang', 'shopcode'], how = 'left')

In [1217]:
# shop_qlkv_region =shop_qlkv_region[['Mã PGD','Miền','QLKV']]

WRITEOFF LIST

In [1218]:
# #  ID writeoff
writeoff_id = '15r30_u_xE0iUexkUgsbZqcZTy0htmSflCp-uty_1hWA'
writeoff_wb = gc.open_by_key(writeoff_id)
write_off_list = to_dataframe(writeoff_wb, 'list',1,0)

In [1219]:
write_off_list = write_off_list[['write_off_period', 'codeno', 'amount']]

In [1220]:
write_off_list.columns

Index(['write_off_period', 'codeno', 'amount'], dtype='object')

DATE MODULE

In [1221]:
day_list = [10,20,25,30]

month_list = [i for i in range(1,13)] 

year_list = [2020]

last_day_of_Feb = calendar.monthrange(year_list[0],month_list[1])

last_day_of_Feb = datetime(year_list[0],month_list[1], last_day_of_Feb[1])

last_day_of_Feb_str = last_day_of_Feb.strftime("%Y-%m-%d")

last_day_of_Feb_str

date_list = [last_day_of_Feb_str]


for l in day_list:
    for k in month_list:
        for i in year_list:
            try:
                sub_date = datetime(i,k,l).strftime("%Y-%m-%d")
                date_list.append(sub_date)
            except:
                pass
date_list.sort()

last_day_list = ['2019-12-31']*4

for z in date_list:
    z_date = datetime.strptime(z, "%Y-%m-%d")
    last_day = calendar.monthrange(z_date.year,z_date.month)[1]
    last_day_of_month = dt.datetime(z_date.year,z_date.month,last_day)
    last_day_of_month = last_day_of_month.strftime("%Y-%m-%d")
    last_day_list.append(last_day_of_month)

last_day_list = last_day_list[:len(last_day_list)-4]

date_dict = {}

for i in range(0, len(date_list)):
    date_dict.update({date_list[i]:last_day_list[i]})
#     sub_date_dict = {last_day_list[i]:date_list[i]}
#     date_dict.update(sub_date_dict)


paired_date_list = []

for i in date_dict.items():
    sub_paired_date_list = [i[1],i[0]]
    paired_date_list.append(sub_paired_date_list)

running_check = dt.date.today()+dt.timedelta(days=-1)

running_check = running_check.strftime("%Y-%m-%d")

# running date list
sub_running_date_list = []

datetime.strptime(running_check, "%Y-%m-%d").month

datetime.strptime(running_check, "%Y-%m-%d").year

for i in range(0,len(paired_date_list)):
    if datetime.strptime(running_check, "%Y-%m-%d").month == datetime.strptime(paired_date_list[i][1], "%Y-%m-%d").month and datetime.strptime(running_check, "%Y-%m-%d").year == datetime.strptime(paired_date_list[i][1], "%Y-%m-%d").year:
        sub_running_date_list.append(paired_date_list[i])

In [1222]:
sub_running_date_list

[['2020-10-31', '2020-11-10'],
 ['2020-10-31', '2020-11-20'],
 ['2020-10-31', '2020-11-25'],
 ['2020-10-31', '2020-11-30']]

In [1223]:
list_date_valid = []

In [22]:
for i in range(0,len(sub_running_date_list)):
    if check_date == sub_running_date_list[i][1]:
        list_date_valid.append(sub_running_date_list[i])

In [23]:
list_date_valid

[['2020-10-31', '2020-11-10']]

ROLLED B0

In [24]:
def rolled_b0(first_date, second_date):
    rolled_b0_str = """Declare @check_1 date = '"""+first_date+"""'
Declare @check_2 date = '"""+second_date+"""'
select a.*, sd.name shopname
from (select pod_1.categoryname,
case when pod_1.shopcode = 'SGN0120' then 'SGN0147'
when pod_1.shopcode = 'SG2046' then 'SGN0126'
when pod_1.shopcode = 'SGN0107' then 'SG2014'
else pod_1.shopcode end shopcode, pod_1.pawnid, pod_1.currentmoney, case when pod_2.tuoino >0 then 1 else 0 end rolled_pawn,
case when pod_2.tuoino >0 then pod_2.currentmoney else 0 end rolled_current_money
from pawnoverdue pod_1
left join pawnoverdue pod_2 on pod_2.pawnid = pod_1.pawnid
where pod_1.created = @check_1
and pod_2.created = @check_2
and pod_1.tuoino<1
and (pod_1.categorycode like '%15%'
or  pod_1.categorycode like '%17%') 
and pod_1.shopcode not like '%HS%' 
and pod_1.shopcode not like '%TEST%'
and pod_1.shopcode not like '%HNIXLN01%'
and pod_1.shopcode not like '%SGNXLN01%') a
left join shopdetail sd on sd.code = a.shopcode
"""
    rolled_b0 = pd.read_sql_query(rolled_b0_str, db)
    rolled_b0_ob = rolled_b0.pivot_table(index = ['shopcode', 'shopname'], columns='categoryname', values= 'pawnid', aggfunc='nunique')
    rolled_b0_cb = rolled_b0.pivot_table(index = ['shopcode', 'shopname'], columns='categoryname', values= 'rolled_pawn', aggfunc='sum')
    rolled_b0_ob_amt = rolled_b0.pivot_table(index = ['shopcode', 'shopname'], columns='categoryname', values= 'currentmoney', aggfunc='sum')
    rolled_b0_cb_amt = rolled_b0.pivot_table(index = ['shopcode', 'shopname'], columns='categoryname', values= 'rolled_current_money', aggfunc='sum')
    rolled_b0 = pd.concat((rolled_b0_ob,rolled_b0_cb,rolled_b0_ob_amt,rolled_b0_cb_amt), axis = 1)
    rolled_b0 = rolled_b0.reset_index()
    rolled_b0.columns = ['shopcode', 'shopname', 'dkxm_all', 'dkot_all', 'dkxm_rolled', 'dkot_rolled','dkxm_all_amt', 'dkot_all_amt', 'dkxm_rolled_amt', 'dkot_rolled_amt']
    rolled_b0 = rolled_b0.fillna(0)
    rolled_b0['rolled_rate_dkxm'] = rolled_b0['dkxm_rolled']/rolled_b0['dkxm_all']
    rolled_b0['rolled_rate_dkxm'] = [float('{:.4}'.format(i)) for i in  rolled_b0['rolled_rate_dkxm']]
    rolled_b0['rolled_rate_dkxm_amt'] = rolled_b0['dkxm_rolled_amt']/rolled_b0['dkxm_all_amt']
    rolled_b0['rolled_rate_dkxm_amt'] = [float('{:.4}'.format(i)) for i in  rolled_b0['rolled_rate_dkxm_amt']]
    rolled_b0['rolled_rate_dkot'] = rolled_b0['dkot_rolled']/rolled_b0['dkot_all']
    rolled_b0['rolled_rate_dkot'] = [float('{:.4}'.format(i)) for i in  rolled_b0['rolled_rate_dkot']]
    rolled_b0['rolled_rate_dkot_amt'] = rolled_b0['dkot_rolled_amt']/rolled_b0['dkot_all_amt']
    rolled_b0['rolled_rate_dkot_amt'] = [float('{:.4}'.format(i)) for i in  rolled_b0['rolled_rate_dkot_amt']]
    rolled_b0['extract_date'] = second_date
    return rolled_b0

In [25]:
rolled_1 = rolled_b0(list_date_valid[0][0], list_date_valid[0][1])

In [26]:
rolled_bo = rolled_1[['extract_date','shopcode','rolled_rate_dkxm_amt','rolled_rate_dkot_amt']]

In [27]:
rolled_bo.columns = ['extract_date','shopcode', 'rolled_rate_motor_title','rolled_rate_car_title']

GROWTH RATE

In [28]:
def pawn_over_due(check_date):
    pod_str = """Declare @check_date date = '"""+check_date+"""'
 select pod.*, datediff(month, sd.opendate, getdate()) shop_age_m
 from (select pawnid, ContractCode , month(created) thang, CategoryName, 
 case when categorycode like '%17%' then 'DKXM'
 when categorycode like '%15%' then 'DKOT'
 else 'SPTT' end 'category_group',case when shopcode = 'SGN0120' then 'SGN0147'
when shopcode = 'SG2046' then 'SGN0126'
when shopcode = 'SGN0107' then 'SG2014'
else shopcode end shopcode,
 case when shopcode = 'SGN0120' then 'SGN0147.409C Lý Thường Kiệt'
when shopcode = 'SG2046' then 'SGN0126.48 Trần Nhân Tôn'
when shopcode = 'SGN0107' then 'F88 HCM - 54H Nguyễn Ảnh Thủ'
else shopname end shopname,  case when TuoiNo<10 then CurrentMoney else 0 end est_net, CurrentMoney
  from PawnOverdue
  where CREATED = @check_date
  and tuoino<10
  and ShopCode not like '%HS%' and
  ShopCode not like '%TEST%'
  and ShopCode not like '%HNIXLN01%' and
  ShopCode not like '%SGNXLN01%') pod
  left join shopdetail sd on sd.code = pod.shopcode"""
    pod_tab = pd.read_sql_query(pod_str, db)
    return pod_tab

In [29]:
def pod_shop(check_date):
    pod_shop = pawn_over_due(check_date).pivot_table(index = ['shopcode','shop_age_m'], columns = ['category_group'], values = 'est_net', aggfunc = 'sum')
    pod_shop = pod_shop.reset_index()
    pod_shop['extract_date'] = check_date
    pod_shop['thang'] = datetime.strptime(check_date, '%Y-%m-%d').month
    return pod_shop

In [30]:
def growth_rate_final(i):
    trial_1 = pod_shop(list_date_valid[i][0])
    trial_2 = pod_shop(list_date_valid[i][1])
    trial_1.columns = ['shopcode', 'shop_age_m', 'start_DKOT', 'start_DKXM', 'start_SPTT', 'start_extract_date', 'start_thang']
    trial_2.columns = ['shopcode', 'shop_age_m', 'end_DKOT', 'end_DKXM', 'end_SPTT', 'end_extract_date', 'end_thang']
    trial = trial_2.merge(trial_1, on = 'shopcode', how = 'left')
    trial = trial.fillna(0)
    trial['DKOT_growth_rate'] = (trial['end_DKOT'] - trial['start_DKOT'])/trial['start_DKOT']
    trial['DKXM_growth_rate'] = (trial['end_DKXM'] - trial['start_DKXM'])/trial['start_DKXM']
    trial['SPTT_growth_rate'] = (trial['end_SPTT'] - trial['start_SPTT'])/trial['start_SPTT']
    return trial

In [31]:
growth_rate = growth_rate_final(0)

In [32]:
#  NUMS GROWTH NET
list_date_valid_growth = []

In [33]:
for i in range(0,len(paired_date_list)):
    if check_date >= paired_date_list[i][1]:
        list_date_valid_growth.append(paired_date_list[i])

In [34]:
list_date_valid_growth.remove(list_date_valid_growth[0])

In [35]:
list_date_valid_growth.reverse()

In [36]:
def growth_result(i):
    pod_shop_lm1 = pod_shop(list_date_valid_growth[i][1])
    pod_shop_lm2 = pod_shop(list_date_valid_growth[i+4][1])
    pod_shop_lm3 = pod_shop(list_date_valid_growth[i+8][1])
    pod_shop_lm4 = pod_shop(list_date_valid_growth[i+12][1])
    pod_shop_lm1.columns = ['shopcode','shop_age_m','dkot(m-1)','dkxm(m-1)','sptt(m-1)','extract_date', 'thang']
    pod_shop_lm2.columns = ['shopcode','shop_age_m2','dkot(m-2)','dkxm(m-2)','sptt(m-2)','extract_date','thang']
    pod_shop_lm3.columns = ['shopcode','shop_age_m3','dkot(m-3)','dkxm(m-3)','sptt(m-3)','extract_date','thang']
    pod_shop_lm4.columns = ['shopcode','shop_age_m4','dkot(m-4)','dkxm(m-4)','sptt(m-4)','extract_date','thang']
    pod_shop_all = pod_shop_lm1.merge(pod_shop_lm2[['shopcode','dkot(m-2)','dkxm(m-2)','sptt(m-2)']], on = 'shopcode', how ='left').merge(pod_shop_lm3[['shopcode','dkot(m-3)','dkxm(m-3)','sptt(m-3)']], on ='shopcode', how = 'left').merge(pod_shop_lm4[['shopcode','dkot(m-4)','dkxm(m-4)','sptt(m-4)']], on ='shopcode', how = 'left')
    pod_shop_all = pod_shop_all.fillna(0)
    pod_shop_all['dkot_diff_m1'] = pod_shop_all['dkot(m-1)'] - pod_shop_all['dkot(m-2)']
    pod_shop_all['dkot_growth_tf1'] = np.where([i>0 for i in pod_shop_all['dkot_diff_m1']],1,0)

    pod_shop_all['dkxm_diff_m1'] = pod_shop_all['dkxm(m-1)'] - pod_shop_all['dkxm(m-2)']
    pod_shop_all['dkxm_growth_tf1'] = np.where([i>0 for i in pod_shop_all['dkxm_diff_m1']],1,0)

    pod_shop_all['sptt_diff_m1'] = pod_shop_all['sptt(m-1)'] - pod_shop_all['sptt(m-2)']
    pod_shop_all['sptt_growth_tf1'] = np.where([i>0 for i in pod_shop_all['sptt_diff_m1']],1,0)
    pod_shop_all['dkot_diff_m2'] = pod_shop_all['dkot(m-2)'] - pod_shop_all['dkot(m-3)']
    pod_shop_all['dkot_growth_tf2'] = np.where([i>0 for i in pod_shop_all['dkot_diff_m2']],1,0)

    pod_shop_all['dkxm_diff_m2'] = pod_shop_all['dkxm(m-2)'] - pod_shop_all['dkxm(m-3)']
    pod_shop_all['dkxm_growth_tf2'] = np.where([i>0 for i in pod_shop_all['dkxm_diff_m2']],1,0)

    pod_shop_all['sptt_diff_m2'] = pod_shop_all['sptt(m-2)'] - pod_shop_all['sptt(m-3)']
    pod_shop_all['sptt_growth_tf2'] = np.where([i>0 for i in pod_shop_all['sptt_diff_m2']],1,0)
    pod_shop_all['dkot_diff_m3'] = pod_shop_all['dkot(m-3)'] - pod_shop_all['dkot(m-4)']
    pod_shop_all['dkot_growth_tf3'] = np.where([i>0 for i in pod_shop_all['dkot_diff_m3']],1,0)

    pod_shop_all['dkxm_diff_m3'] = pod_shop_all['dkxm(m-3)'] - pod_shop_all['dkxm(m-4)']
    pod_shop_all['dkxm_growth_tf3'] = np.where([i>0 for i in pod_shop_all['dkxm_diff_m3']],1,0)

    pod_shop_all['sptt_diff_m3'] = pod_shop_all['sptt(m-3)'] - pod_shop_all['sptt(m-4)']
    pod_shop_all['sptt_growth_tf3'] = np.where([i>0 for i in pod_shop_all['sptt_diff_m3']],1,0)
    pod_shop_all['dkot_growth_rate'] =(pod_shop_all['dkot(m-1)'] - pod_shop_all['dkot(m-2)'])/pod_shop_all['dkot(m-2)']

    pod_shop_all['dkot_growth_rate'] = ['{0:.3g}'.format(i) for i in pod_shop_all['dkot_growth_rate']]

    pod_shop_all['dkxm_growth_rate'] = (pod_shop_all['dkxm(m-1)']-pod_shop_all['dkxm(m-2)'])/pod_shop_all['dkxm(m-2)']

    pod_shop_all['dkxm_growth_rate'] = ['{0:.3g}'.format(i) for i in pod_shop_all['dkxm_growth_rate']]

    pod_shop_all['sptt_growth_rate'] = (pod_shop_all['sptt(m-1)'] - pod_shop_all['sptt(m-2)'])/pod_shop_all['sptt(m-2)']

    pod_shop_all['sptt_growth_rate'] = ['{0:.3g}'.format(i) for i in pod_shop_all['sptt_growth_rate']]

    pod_shop_all['dkot_growth_count'] = pod_shop_all['dkot_growth_tf1']+pod_shop_all['dkot_growth_tf2']+pod_shop_all['dkot_growth_tf3']

    pod_shop_all['dkxm_growth_count'] = pod_shop_all['dkxm_growth_tf1']+pod_shop_all['dkxm_growth_tf2']+pod_shop_all['dkxm_growth_tf3']

    pod_shop_all['sptt_growth_count'] = pod_shop_all['sptt_growth_tf1']+pod_shop_all['sptt_growth_tf2']+pod_shop_all['sptt_growth_tf3']

    pod_shop_all['total_growth_count'] = pod_shop_all['dkot_growth_count'] + pod_shop_all['dkxm_growth_count'] + pod_shop_all['sptt_growth_count']
    return pod_shop_all

In [37]:
growth_count_all = growth_result(0)

In [38]:
growth_count_all = growth_count_all[['shopcode','thang','dkot_growth_count', 'dkxm_growth_count', 'sptt_growth_count', 'total_growth_count']]

In [39]:
growth_rate = growth_rate.merge(growth_count_all,left_on = ['end_thang', 'shopcode'],right_on = ['thang','shopcode'], how = 'left' )

In [40]:
growth_rate = growth_rate[['end_extract_date','end_thang','shopcode','DKOT_growth_rate', 
                           'DKXM_growth_rate', 'SPTT_growth_rate',
                           'dkot_growth_count', 'dkxm_growth_count', 
                           'sptt_growth_count',
       'total_growth_count']]

In [343]:
growth_rate.columns = ['extract_date','thang','shopcode', 'growth_rate_car_title', 
                       'growth_rate_motor_title','growth_rate_normal_asset', 
                       'nums_grownet_cartitle','nums_grownet_motortitle', 'nums_grownet_normalasset', 'total_nums_growthnet']

BALANCE RATIO

In [41]:
def balance_ratio(todate):
    balance_ratio_str = """Declare @check_date date = '"""+todate+"""'
select created, pawnid,
case when categorycode like '%15%' or categorycode like '%17%' then categoryname
else 'SPTT' end categoryname, contractcode, 
case when shopcode = 'SGN0120' then 'SGN0147'
when shopcode = 'SG2046' then 'SGN0126'
when shopcode = 'SGN0107' then 'SG2014'
else shopcode end shopcode,
 case when shopcode = 'SGN0120' then 'SGN0147.409C Lý Thường Kiệt'
when shopcode = 'SG2046' then 'SGN0126.48 Trần Nhân Tôn'
when shopcode = 'SGN0107' then 'F88 HCM - 54H Nguyễn Ảnh Thủ'
else shopname end shopname, case when tuoino <=0 then '0. dpd<=0'
when tuoino <=10 and tuoino>0 then '1. 0<dpd<10'
when tuoino <=90 and tuoino >10 then '2. 10<=dpd<90'
else '3. dpd>=90' end nhom_no, currentmoney
from pawnoverdue
where created = @check_date
and shopcode not like '%HS%'
and shopcode not like '%TEST%'
and shopcode not like '%HNIXLN01%'
and shopcode not like '%SGNXLN01%'"""
    balance_ratio_raw = pd.read_sql_query(balance_ratio_str, db)
    balance_ratio_raw = balance_ratio_raw.merge(write_off_list[['codeno', 'amount']], left_on = 'contractcode', right_on = 'codeno', how = 'left')
    balance_ratio_raw = balance_ratio_raw[balance_ratio_raw['codeno'].isnull() == True]
    bal_dpd_ratio = balance_ratio_raw.fillna(0)
    balance_ratio_dpd = bal_dpd_ratio.pivot_table(index = ['categoryname','shopcode'], columns='nhom_no', values='currentmoney', aggfunc='sum')
    balance_ratio_dpd = balance_ratio_dpd.fillna(0) 
    balance_ratio_dpd = balance_ratio_dpd.reset_index()
    balance_ratio_dpd['total_balance'] = balance_ratio_dpd['0. dpd<=0']+balance_ratio_dpd['1. 0<dpd<10']+balance_ratio_dpd['2. 10<=dpd<90']+balance_ratio_dpd['3. dpd>=90']
    balance_ratio_dpd['0. dpd<=0_rate'] = balance_ratio_dpd['0. dpd<=0']/balance_ratio_dpd['total_balance']
    balance_ratio_dpd['1. 0<dpd<10_rate'] = balance_ratio_dpd['1. 0<dpd<10']/balance_ratio_dpd['total_balance']
    balance_ratio_dpd['2. 10<=dpd<90_rate'] = balance_ratio_dpd['2. 10<=dpd<90']/balance_ratio_dpd['total_balance']
    balance_ratio_dpd['3. dpd>=90_rate'] = balance_ratio_dpd['3. dpd>=90']/balance_ratio_dpd['total_balance']
    balance_ratio_dpd['extract_date'] = todate
#     balance_ratio_raw = balance_ratio_raw.merge(shop_data[['Mã PGD','GĐM', 'TPK', 'QLKV', 'Miền', 'Tỉnh/ Thành phố','khu_vuc']], left_on = 'shopcode', right_on = 'Mã PGD', how = 'left')
    return balance_ratio_dpd

In [42]:
balance_ratio_m = balance_ratio(list_date_valid[0][1])

In [43]:
balance_ratio = balance_ratio_m

In [44]:
balance_ratio['0.dpd<11'] = balance_ratio['0. dpd<=0']+balance_ratio['1. 0<dpd<10']

In [45]:
balance_ratio_cartitle = balance_ratio[balance_ratio['categoryname']=='Đăng ký Ô tô'][['extract_date','shopcode','0. dpd<=0_rate', '1. 0<dpd<10_rate', '2. 10<=dpd<90_rate', '3. dpd>=90_rate','0.dpd<11' ]]

In [46]:
balance_ratio_cartitle.columns = ['extract_date','shopcode', '0.cartitle_dpd<=0_rate', '1.cartitle_0<dpd<10_rate', '2.cartitle_10<=dpd<90_rate','3.cartitle_dpd>=90_rate','cartt_0.dpd<11']

In [47]:
balance_ratio_motortitle = balance_ratio[balance_ratio['categoryname']=='Đăng ký xe máy'][['extract_date','shopcode','0. dpd<=0_rate', '1. 0<dpd<10_rate', '2. 10<=dpd<90_rate', '3. dpd>=90_rate','0.dpd<11' ]]

In [48]:
balance_ratio_motortitle.columns = ['extract_date','shopcode', '0.motortitle_dpd<=0_rate', '1.motortitle_0<dpd<10_rate', '2.motortitle_10<=dpd<90_rate','3.motortitle_dpd>=90_rate','motortt_0.dpd<11']

In [49]:
balance_ratio_normal_asset = balance_ratio[balance_ratio['categoryname']=='SPTT'][['extract_date','shopcode','0. dpd<=0_rate', '1. 0<dpd<10_rate', '2. 10<=dpd<90_rate', '3. dpd>=90_rate','0.dpd<11' ]]

In [50]:
balance_ratio_normal_asset.columns = ['extract_date','shopcode', '0.normass_dpd<=0_rate', '1.normass_0<dpd<10_rate', '2.normass_10<=dpd<90_rate','3.normass_dpd>=90_rate','normass_0.dpd<11']

In [51]:
def pawn_balance_ratio(todate):
    balance_ratio_str = """Declare @check_date date = '"""+todate+"""'
select created, pawnid,
case when categorycode like '%15%' or categorycode like '%17%' then categoryname
else 'SPTT' end categoryname, contractcode, 
case when shopcode = 'SGN0120' then 'SGN0147'
when shopcode = 'SG2046' then 'SGN0126'
when shopcode = 'SGN0107' then 'SG2014'
else shopcode end shopcode,
 case when shopcode = 'SGN0120' then 'SGN0147.409C Lý Thường Kiệt'
when shopcode = 'SG2046' then 'SGN0126.48 Trần Nhân Tôn'
when shopcode = 'SGN0107' then 'F88 HCM - 54H Nguyễn Ảnh Thủ'
else shopname end shopname, case when tuoino <=0 then '0. dpd<=0'
when tuoino <=10 and tuoino>0 then '1. 0<dpd<10'
when tuoino <=90 and tuoino >10 then '2. 10<=dpd<90'
else '3. dpd>=90' end nhom_no, currentmoney
from pawnoverdue
where created = @check_date
and shopcode not like '%HS%'
and shopcode not like '%TEST%'
and shopcode not like '%HNIXLN01%'
and shopcode not like '%SGNXLN01%'"""
    balance_ratio_raw = pd.read_sql_query(balance_ratio_str, db)
    balance_ratio_raw = balance_ratio_raw.merge(write_off_list[['codeno', 'amount']], left_on = 'contractcode', right_on = 'codeno', how = 'left')
    balance_ratio_raw = balance_ratio_raw[balance_ratio_raw['codeno'].isnull() == True]
    bal_dpd_ratio = balance_ratio_raw.fillna(0)
    balance_ratio_dpd = bal_dpd_ratio.pivot_table(index = ['categoryname','shopcode'], columns='nhom_no', values='pawnid', aggfunc='nunique')
    balance_ratio_dpd = balance_ratio_dpd.fillna(0) 
    balance_ratio_dpd = balance_ratio_dpd.reset_index()
    balance_ratio_dpd['total_pawn'] = balance_ratio_dpd['0. dpd<=0']+balance_ratio_dpd['1. 0<dpd<10']+balance_ratio_dpd['2. 10<=dpd<90']+balance_ratio_dpd['3. dpd>=90']
    balance_ratio_dpd['0. dpd<=0_rate'] = balance_ratio_dpd['0. dpd<=0']/balance_ratio_dpd['total_pawn']
    balance_ratio_dpd['1. 0<dpd<10_rate'] = balance_ratio_dpd['1. 0<dpd<10']/balance_ratio_dpd['total_pawn']
    balance_ratio_dpd['2. 10<=dpd<90_rate'] = balance_ratio_dpd['2. 10<=dpd<90']/balance_ratio_dpd['total_pawn']
    balance_ratio_dpd['3. dpd>=90_rate'] = balance_ratio_dpd['3. dpd>=90']/balance_ratio_dpd['total_pawn']
    balance_ratio_dpd['extract_date'] = todate
#     balance_ratio_raw = balance_ratio_raw.merge(shop_data[['Mã PGD','GĐM', 'TPK', 'QLKV', 'Miền', 'Tỉnh/ Thành phố','khu_vuc']], left_on = 'shopcode', right_on = 'Mã PGD', how = 'left')
    return balance_ratio_dpd

In [52]:
balance_ratio_m_pawn = pawn_balance_ratio(list_date_valid[0][1])

In [53]:
balance_ratio_m_pawn.columns = ['categoryname', 'shopcode', 'num_pawn_0. dpd<=0', 'num_pawn_1. 0<dpd<10', 'num_pawn_2. 10<=dpd<90',
       'num_pawn_3. dpd>=90', 'total_pawn', 'num_pawn_0. dpd<=0_rate', 'num_pawn_1. 0<dpd<10_rate',
       'num_pawn_2. 10<=dpd<90_rate', 'num_pawn_3. dpd>=90_rate','extract_date']

In [54]:
balance_ratio_pawn = balance_ratio_m_pawn

In [55]:
balance_ratio_cartitle_pawn = balance_ratio_pawn[balance_ratio_pawn['categoryname']=='Đăng ký Ô tô'][['extract_date','shopcode', 'num_pawn_0. dpd<=0','num_pawn_1. 0<dpd<10', 'num_pawn_2. 10<=dpd<90', 'num_pawn_3. dpd>=90','total_pawn', 'num_pawn_0. dpd<=0_rate', 'num_pawn_1. 0<dpd<10_rate','num_pawn_2. 10<=dpd<90_rate', 'num_pawn_3. dpd>=90_rate']]

balance_ratio_cartitle_pawn.columns = ['extract_date','shopcode', 'cartt_num_pawn_0. dpd<=0', 'cartt_num_pawn_1. 0<dpd<10',
       'cartt_num_pawn_2. 10<=dpd<90', 'cartt_num_pawn_3. dpd>=90', 'cartt_total_pawn',
       'cartt_num_pawn_0. dpd<=0_rate', 'cartt_num_pawn_1. 0<dpd<10_rate',
       'cartt_num_pawn_2. 10<=dpd<90_rate', 'cartt_num_pawn_3. dpd>=90_rate']

In [56]:
balance_ratio_motortitle_pawn = balance_ratio_pawn[balance_ratio_pawn['categoryname']=='Đăng ký xe máy'][['extract_date','shopcode', 'num_pawn_0. dpd<=0','num_pawn_1. 0<dpd<10', 'num_pawn_2. 10<=dpd<90', 'num_pawn_3. dpd>=90','total_pawn', 'num_pawn_0. dpd<=0_rate', 'num_pawn_1. 0<dpd<10_rate','num_pawn_2. 10<=dpd<90_rate', 'num_pawn_3. dpd>=90_rate']]

balance_ratio_motortitle_pawn.columns = ['extract_date','shopcode', 'motortt_num_pawn_0. dpd<=0', 'motortt_num_pawn_1. 0<dpd<10',
       'motortt_num_pawn_2. 10<=dpd<90', 'motortt_num_pawn_3. dpd>=90', 'motortt_total_pawn',
       'motortt_num_pawn_0. dpd<=0_rate', 'motortt_num_pawn_1. 0<dpd<10_rate',
       'motortt_num_pawn_2. 10<=dpd<90_rate', 'motortt_num_pawn_3. dpd>=90_rate']

FAV

In [57]:
def fav_raw(fromdate, todate):
    fav_data_str = """Declare @fromdate date ='"""+fromdate+"""'
    Declare @todate date = '"""+todate+"""'
select case when sd.code = 'SGN0120' then 'SGN0147'
when sd.code = 'SG2046' then 'SGN0126'
when sd.code = 'SGN0107' then 'SG2014'
else sd.code end shopcode,
 case when sd.code = 'SGN0120' then 'SGN0147.409C Lý Thường Kiệt'
when sd.code = 'SG2046' then 'SGN0126.48 Trần Nhân Tôn'
when sd.code = 'SGN0107' then 'F88 HCM - 54H Nguyễn Ảnh Thủ'
else sd.name end shopname, lsf.[PAWNID] ,lsf.[CODE_NO],lsf.[ASSET_NAME], cast(lsf.[SALE_DATE] as date) sale_date, lsf.[MONEY_APPRAISAL], lsf.[SALE_PRICE]
from [dwh].[dbo].[W_LIQUID_SALE_F] lsf
left join (select distinct(PO_ITEMS_ID) distinct_p 
from [dwh].[dbo].[W_LIQUID_SALE_F]
where sale_price <0) lsf2 on lsf2.distinct_p = lsf.pawnid
left join shopdetail sd on sd.shopid = lsf.[SHOP_WID]
where cast(lsf.[SALE_DATE] as date) >@fromdate
and cast(lsf.[SALE_DATE] as date) <= @todate
and lsf2.distinct_p is null
and sd.code not like '%TEST%'
and sd.code not like '%HS%'
and sd.code not like '%HNIXLN01%'
and sd.code not like '%SGNXLN01%'
"""
    fav_data = pd.read_sql_query(fav_data_str, db)
#     fav_data = fav_data.merge(shop_data[['Mã PGD', 'GĐM', 'TPK', 'QLKV', 'Miền', 'Tỉnh/ Thành phố']], left_on = 'shopcode', right_on = 'Mã PGD', how = 'left')
    fav_shop = fav_data.groupby('shopname').agg({'sale_price':'sum', 'money_appraisal':'sum'}).reset_index()
#     fav_shop = fav_shop.fillna(0)
    fav_shop['fav_rate'] = fav_shop['sale_price']/fav_shop['money_appraisal']
    fav_shop['extract_date'] = todate
    return fav_shop

In [58]:
fav_m = fav_raw(list_date_valid[0][0], list_date_valid[0][1])

In [59]:
fav = fav_m

In [213]:
fav['extract_date'] = pd.to_datetime(fav['extract_date'], format="%Y-%m-%d")

ONTIME

In [60]:
n_days_after = '3'

In [61]:
write_off_list.columns = ['write_off_period', 'pawn_code', 'amount']

In [62]:
# đk Ô tô
def ontime_dkot(fromdate, todate, n_days_after):
    monthly_ontime_str = """Declare @schedule_from date ='"""+fromdate+"""'
Declare @schedule_to date ='"""+todate+"""'
select 'monthly_org_paid' title_type, *,case when (pmt_stt like N'overdue' and dpd_"""+n_days_after+"""_day_after is not null and dpd_"""+n_days_after+"""_day_after>0) then N'still_over_due'
when pmt_stt like N'overdue' and (dpd_"""+n_days_after+"""_day_after is null or dpd_"""+n_days_after+"""_day_after=0) and t_"""+n_days_after+"""_day_after<getdate() then N'paid_after_"""+n_days_after+"""_day'
when pmt_stt like N'overdue' and t_"""+n_days_after+"""_day_after>=getdate() then N'out_of_range'
else N'ontime' end as stt_after_"""+n_days_after+"""_day,
case when PackageCode like '%VAY_NHANH%' then N'Vay nhanh' 
else N'Vay thường' end as package_type
 from (
select  pd.pawnid,po.tuoino dpd_yesterday,po2.TuoiNo dpd_"""+n_days_after+"""_day_after,pa.PackageCode,pa.shopname, pa.shopcode,areaid,
dbo.getyearmonth(pd.scheduletodate)ym,pd.scheduletodate,
dateadd(day,"""+n_days_after+""",pd.ScheduleToDate) t_"""+n_days_after+"""_day_after,pd.acc_sum,pd.paydate,
pd.acc_prin_payment,paidmoney,pa.status,pa.CodeNo,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt, pa.cus_type,  
case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=(acc_sum-1000)) or CloseDate<=ScheduleToDate 
then N'ontime' else N'overdue' end as pmt_stt,
case when pa.frequency <= 30 then N'1. Kỳ hạn 1 tháng'
  when pa.frequency <= 90 then N'2. Kỳ hạn 3 tháng'
  when pa.frequency <= 180 then N'3. Kỳ hạn 6 tháng'
  when pa.frequency <= 270 then N'4. Kỳ hạn 9 tháng'
  when pa.frequency <= 360 then N'5. Kỳ hạn 12 tháng'
  else N'6. >12 tháng' end package_time_range,
  case when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 50000000 then '0. <=50tr'
when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 100000000 then '2. 100-200tr'
when pa.PackageCode is not null and  (pa.LoanMoneyOrg - pa.[InsurranceMoney]) > 200000000 then N'4. >200tr'
when pa.PackageCode is null and  (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 300000000 then '3. 200-300tr'
else N'4. >300tr' end "package_money_range"
from pawnoverduedetail pd  
left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))
left join pawnoverdue po2 on (po2.pawnid=pd.pawnid and po2.created=dateadd(day,"""+n_days_after+""",pd.scheduletodate))
left join  (select pa1.*,shop.name shopname,shop.areaid,
case when Lag (Pawnid) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) is null then N'Khách mới' else N'Khách cũ' end "cus_type" from Pawn pa1 
left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid 
left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid  
 where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt)) 
 and pa.STATUS not in (15,11,140) and pa.CategoryCode like N'%15%'
  and InterestMoney like N'%goc%' and pd.scheduletodate<@schedule_to and pd.scheduletodate>@schedule_from) tab
  where dpd_yesterday<=10
  order by pawnid,scheduletodate"""
    eop_ontime_str= """Declare @schedule_from date ='"""+fromdate+"""'
Declare @schedule_to date ='"""+todate+"""'
select 'eop_org_paid' title_type, *,case when (pmt_stt like N'overdue' and dpd_"""+n_days_after+"""_day_after is not null and dpd_"""+n_days_after+"""_day_after>0) then N'still_over_due'
when pmt_stt like N'overdue' and (dpd_"""+n_days_after+"""_day_after is null or dpd_"""+n_days_after+"""_day_after=0) and t_"""+n_days_after+"""_day_after<getdate() then N'paid_after_"""+n_days_after+"""_day'
when pmt_stt like N'overdue' and t_"""+n_days_after+"""_day_after>=getdate() then N'out_of_range'
else N'ontime' end as stt_after_"""+n_days_after+"""_day,
case when PackageCode like '%VAY_NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type
 from (
select  ROW_NUMBER() over (partition by pd.pawnid order by moneyinterest) as row_num,pd.pawnid,po.tuoino dpd_yesterday,
po2.TuoiNo dpd_"""+n_days_after+"""_day_after,pa.PackageCode,pa.shopname,pa.shopcode,areaid,dbo.getyearmonth(pd.scheduletodate)ym,pd.scheduletodate,dateadd(day,"""+n_days_after+""",pd.ScheduleToDate) t_"""+n_days_after+"""_day_after
,pd.acc_sum,pd.paydate,
pd.acc_prin_payment,paidmoney,pa.status,pa.CodeNo,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt, pa.cus_type,  
case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=(acc_sum-1000)) or CloseDate<=ScheduleToDate 
then N'ontime' else N'overdue' end as pmt_stt,
case when pa.frequency <= 30 then N'1. Kỳ hạn 1 tháng'
  when pa.frequency <= 90 then N'2. Kỳ hạn 3 tháng'
  when pa.frequency <= 180 then N'3. Kỳ hạn 6 tháng'
  when pa.frequency <= 270 then N'4. Kỳ hạn 9 tháng'
  when pa.frequency <= 360 then N'5. Kỳ hạn 12 tháng'
  else N'6. >12 tháng' end package_time_range,
  case when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 50000000 then '0. <=50tr'
when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 100000000 then '2. 100-200tr'
when pa.PackageCode is not null and  (pa.LoanMoneyOrg - pa.[InsurranceMoney]) > 200000000 then N'4. >200tr'
when pa.PackageCode is null and  (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 300000000 then '3. 200-300tr'
else N'4. >300tr' end "package_money_range"
from pawncaroverdue pd  
left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))
left join pawnoverdue po2 on (po2.pawnid=pd.pawnid and po2.created=dateadd(day,"""+n_days_after+""",pd.scheduletodate))
left join  (select pa1.*,shop.name shopname,shop.areaid,
case when Lag (Pawnid) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) is null then N'Khách mới' else N'Khách cũ' end "cus_type"
from Pawn pa1 
left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid 
left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid  
 where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt)) 
 and pa.STATUS not in (15,11,140) and pa.CategoryCode like N'%15%'
  and pd.scheduletodate<=@schedule_to and pd.scheduletodate>@schedule_from) tab
  where dpd_yesterday<=10 and row_num=1
  order by pawnid,scheduletodate"""
    monthly_ontime = pd.read_sql_query(monthly_ontime_str,db)
    eop_ontime = pd.read_sql_query(eop_ontime_str,db)
    ontime_data = pd.concat([monthly_ontime, eop_ontime], ignore_index=True)
    ontime_data = ontime_data.merge(write_off_list, left_on = 'codeno', right_on = 'pawn_code',how = 'left')
    ontime_data = ontime_data[ontime_data['pawn_code'].isnull()==True]
    column_name='stt_after_'+n_days_after+'_day'
    colmn_name_2='paid_after_'+n_days_after+'_day'
#     ontime_data=pd.merge(ontime_data,shop_data, left_on='shopcode',right_on='Mã PGD',how='left')
#     ontime_data = ontime_data[ontime_data['Vùng/Miền'].isnull() == False]
#     ontime_data['khu_vuc'] = np.where(ontime_data['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(ontime_data['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((ontime_data['Tỉnh/ Thành phố'] != 'Hà Nội') & (ontime_data['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((ontime_data['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (ontime_data['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN', 'Miền Trung'))))
    ontime_data.insert(0,'categoryname','Đăng ký Ô tô')
    ontime_data['brr_ym'] = [i.strftime("%Y%m") for i in ontime_data['fromdate']]
    ontime_car_shop = ontime_data.pivot_table(index='shopname', columns='pmt_stt', values='pawnid', aggfunc='nunique').reset_index()
    ontime_car_shop = ontime_car_shop.fillna(0)
    ontime_car_shop['total'] =ontime_car_shop['ontime']+ontime_car_shop['overdue']
    ontime_car_shop['cartitle_ontime_rate'] = ontime_car_shop['ontime']/ontime_car_shop['total']
    ontime_car_shop['extract_date'] = todate
    return ontime_car_shop

In [63]:
def ontime_dkxm(fromdate,todate,n_days_after_num):
    n_days_after=n_days_after_num
    ontime_dkxm_str = """Declare @schedule_from date ='"""+fromdate+"""'
Declare @schedule_to date ='"""+todate+"""'
select 'motor_title' title_type,*,case when (pmt_stt like N'overdue' and dpd_"""+n_days_after+"""_day_after is not null and dpd_"""+n_days_after+"""_day_after>0) then N'still_over_due'
when pmt_stt like N'overdue' and (dpd_"""+n_days_after+"""_day_after is null or dpd_"""+n_days_after+"""_day_after=0) and t_"""+n_days_after+"""_day_after<getdate() then N'paid_after_"""+n_days_after+"""_day'
when pmt_stt like N'overdue' and t_"""+n_days_after+"""_day_after>=getdate() then N'out_of_range'
else N'ontime' end as stt_after_"""+n_days_after+"""_day,case when PackageCode like '%VAY_NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type
 from (
select  pd.pawnid,po.tuoino dpd_yesterday,po2.TuoiNo dpd_"""+n_days_after+"""_day_after,pa.PackageCode, pa.shopname,pa.shopcode,
areaid,dbo.getyearmonth(pd.scheduletodate)ym,pd.scheduletodate,dateadd(day,"""+n_days_after_num+""",pd.ScheduleToDate) t_"""+n_days_after+"""_day_after,pd.principalmoney,pd.acc_sum,pd.paydate,
pd.acc_prin_payment,paidmoney,pa.status,pa.CodeNo,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt,  pa.cus_type,
case  when (pd.paydate<=ScheduleToDate  and
acc_prin_payment>=acc_sum) or CloseDate<=ScheduleToDate 
then N'ontime' else N'overdue' end as pmt_stt,
case when pa.frequency <= 30 then N'1. Kỳ hạn 1 tháng'
  when pa.frequency <= 90 then N'2. Kỳ hạn 3 tháng'
  when pa.frequency <= 180 then N'3. Kỳ hạn 6 tháng'
  when pa.frequency <= 270 then N'4. Kỳ hạn 9 tháng'
  when pa.frequency <= 360 then N'5. Kỳ hạn 12 tháng'
  else N'6. >12 tháng' end package_time_range,
  case when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 3000000 then '0. <=3tr'
		when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 10000000 then '1. 3-10tr'
		when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 15000000 then '2. 10-15tr'
		when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 25000000 then '3. 15-25tr'
		else '4. >25tr' end "package_money_range"
from pawnoverduedetail pd  
left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))
left join pawnoverdue po2 on (po2.pawnid=pd.pawnid and po2.created=dateadd(day,"""+n_days_after_num+""",pd.scheduletodate))
left join  (select pa1.*,shop.name shopname,shop.areaid,
case when Lag (pa1.Pawnid) over (PARTITION BY pa1.customerid, pa1.[CategoryCode] ORDER BY pa1.fromdate asc) is null then N'Khách mới' else N'Khách cũ' end "cus_type" from Pawn pa1 
left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid 
left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid  
 where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt)) 
 and pa.STATUS not in (15,11,140) and pa.CategoryCode like N'%17%'
  and InterestMoney like N'%goc%' and pd.scheduletodate<=@schedule_to and pd.scheduletodate>@schedule_from) tab
  where dpd_yesterday<=10
  order by pawnid,scheduletodate"""
#     return ontime_dkxm_str
    ontime_data=pd.read_sql_query(ontime_dkxm_str,db)
    ontime_data = ontime_data.merge(write_off_list, left_on = 'codeno', right_on = 'pawn_code',how = 'left')
    ontime_data = ontime_data[ontime_data['pawn_code'].isnull()==True]
    column_name='stt_after_'+n_days_after+'_day'
    colmn_name_2='paid_after_'+n_days_after+'_day'
#     ontime_data=pd.merge(ontime_data,shop_data,left_on='shopcode',right_on='Mã PGD',how='left')
#     ontime_data = ontime_data[ontime_data['Vùng/Miền'].isnull() == False]
#     ontime_data['khu_vuc'] = np.where(ontime_data['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(ontime_data['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((ontime_data['Tỉnh/ Thành phố'] != 'Hà Nội') & (ontime_data['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((ontime_data['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (ontime_data['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN','Miền Trung'))))
    ontime_data.insert(0,'categoryname','Đăng ký xe máy')
    ontime_data['brr_ym'] = [i.strftime("%Y%m") for i in ontime_data['fromdate']]
    ontime_motor_shop = ontime_data.pivot_table(index='shopname', columns='pmt_stt', values='pawnid', aggfunc='nunique').reset_index()
    ontime_motor_shop = ontime_motor_shop.fillna(0)
    ontime_motor_shop['total'] =ontime_motor_shop['ontime']+ontime_motor_shop['overdue']
    ontime_motor_shop['motortitle_ontime_rate'] = ontime_motor_shop['ontime']/ontime_motor_shop['total']
    ontime_motor_shop['extract_date'] = todate
    #     ontime_pivot=pd.pivot_table(ontime_data,ikto máy')=['areaid','shopname', 'Miền', 'Tỉnh/ Thành phố','package_type'],columns=column_name,values='pawnid',aggfunc='count',margins=True).reset_index()
#     ontime_pivot.insert(0,'categoryname','Đăng ký xe máy')
#     ontime_pivot['ontime_rate']=ontime_pivot.ontime/ontime_pivot.All
#     ontime_pivot['paid_when_dpd_less_than_'+n_days_after_num+'_rate']=(ontime_pivot.ontime+ontime_pivot[colmn_name_2])/ontime_pivot.All
#     ontime_pivot.to_excel(r'D:\F88\Python code\ontime_dkot\ontime_dkxm_'+fromdate+"_"+todate+'.xlsx', index=False)
    
#     ontime_data['todate_ym'] = datetime.strptime(ontime['todate'], '%Y-%m-%d').strftime('%Y%m')
#     ontime_data['fromdate_ym'] = datetime.strptime(ontime['fromdate'], '%Y-%m-%d').strftime('%Y%m')
    return ontime_motor_shop

In [64]:
ontime_dkot_this_month = ontime_dkot(list_date_valid[0][0],list_date_valid[0][1],n_days_after)
ontime_dkxm_this_month = ontime_dkxm(list_date_valid[0][0],list_date_valid[0][1],n_days_after)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [65]:
ontime_motortiltle_rate = ontime_dkxm_this_month

In [66]:
ontime_cartitle_rate = ontime_dkot_this_month

EARLY REMIND

In [67]:
def remind_rate_cartitle(fromdate, todate):
    remind_cartitle = """Declare @schedule_from date ='"""+fromdate+"""'
                Declare @schedule_to date ='"""+todate+"""'
                select  tab.*,cmt.*,
DATEDIFF(day,cmt.cmtdate,ScheduleToDate) day_cmt_before_schedule,
case when cmtdate is null then N'1.no_cmt'
when DATEDIFF(day,cmt.cmtdate,ScheduleToDate)=0 then N'3.on_due_date'
when DATEDIFF(day,cmt.cmtdate,ScheduleToDate)<>0 then N'2.early_remind' end as cmt_type,
case when PackageCode like '%NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type
from ( select  pd.pawnid,po.tuoino dpd_yesterday,
pa.PackageCode,pa.shopname, pa.shopcode,
areaid,pd.scheduletodate,pd.paydate, pd.acc_prin_payment,
pa.status,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt,   
case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=acc_sum) or CloseDate<=ScheduleToDate 
then N'ontime' else N'overdue' end as pmt_stt, pd.type_cartitle
from (select 'monthly_paid_origin' type_cartitle, podd.[PawnID],podd.[ScheduleToDate] ,podd.[ScheduleFromDate]
      ,podd.[acc_prin_payment],podd.[paydate] ,podd.[InterestMoney], podd.acc_sum from [dwh].[dbo].[PawnOverDuedetail] podd
	  left join pawn pa on pa.pawnid = podd.pawnid
	  where InterestMoney like N'%goc%' and podd.scheduletodate<=@schedule_to and podd.scheduletodate>@schedule_from
	  and pa.categorycode like '%15%'
	  union all
	  (select 'eop_paid_origin' type_cartitle, [PawnID]
      ,[ScheduleToDate]
      ,[ScheduleFromDate]
      ,[acc_prin_payment]
      ,[paydate]
      ,[MoneyInterest] as [InterestMoney],
	  acc_sum
  FROM [dwh].[dbo].[pawnCarOverDue]
  where [MoneyInterest] like '%lai%' and scheduletodate<=@schedule_to and scheduletodate>=@schedule_from)) pd 
   left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))  
   left join  (select pa1.*,shop.name shopname,shop.areaid from Pawn pa1 
   left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid  
   left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid   
where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt))  
and pa.STATUS not in (15,11,140) ) tab
  left join (select pawnid pawn_wid,CodeNo,Body,CommentType,cmtdate,CreatedDate cmt_dt from CommentContract 
  where PGD is not null and cmtdate>= dateadd(day,-6,@schedule_from) and cmtdate<=@schedule_to)cmt 
  on cmt.pawn_wid=tab.pawnid and cmt.cmtdate>= dateadd(day,-6,tab.scheduletodate) and cmt.cmtdate<= tab.ScheduleToDate
 where dpd_yesterday<=5 
 order by tab.PawnID"""
    remind_stats = pd.read_sql_query(remind_cartitle,db)
    remind_stats = remind_stats.merge(write_off_list, left_on='codeno', right_on='pawn_code',how = 'left')
    remind_stats = remind_stats[remind_stats['pawn_code'].isnull()==True]
#     remind_stats = remind_stats.merge(shop_data, left_on='shopname', right_on = 'Tên PGD', how = 'left')
#     remind_stats['khu_vuc'] = np.where(remind_stats['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(remind_stats['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((remind_stats['Tỉnh/ Thành phố'] != 'Hà Nội') & (remind_stats['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((remind_stats['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (remind_stats['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN', 'Miền Trung'))))
    need_remind = remind_stats.groupby('shopname').agg({'pawnid': 'nunique'}).reset_index()
    remind_early = remind_stats.pivot_table(index = 'shopname', columns='cmt_type', values='pawnid', aggfunc='nunique').reset_index()
    remind_early = remind_early.merge(need_remind, on = 'shopname', how='left')
    remind_early = remind_early.fillna(0)
    remind_early['early_rate_remind'] = remind_early['2.early_remind']/remind_early['pawnid']
    remind_early['ondue_rate_remind'] = remind_early['3.on_due_date']/remind_early['pawnid']
    remind_early['extract_date'] = todate
    return remind_early

In [68]:
remind_stats_cartitle = remind_rate_cartitle(list_date_valid[0][0],list_date_valid[0][1])

In [69]:
def remind_rate(fromdate, todate):
    remind_str =  """Declare @schedule_from date ='"""+fromdate+"""'
                Declare @schedule_to date ='"""+todate+"""'
select tab.*,cmt.*,DATEDIFF(day,cmtdate,ScheduleToDate) day_cmt_before_schedule,
  case when cmtdate is null then N'1.no_cmt'
  when DATEDIFF(day,cmtdate,ScheduleToDate)=0 then N'3.on_due_date'
   when DATEDIFF(day,cmtdate,ScheduleToDate)<>0 then N'2.early_remind' end as cmt_type,
 case when PackageCode like '%NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type
 from ( select  pd.pawnid,po.tuoino dpd_yesterday,
 pa.PackageCode,pa.shopname, pa.shopcode,
 areaid,dbo.getyearmonth(pd.scheduletodate)ym,pd.scheduletodate,pd.principalmoney,pd.acc_sum,pd.paydate, pd.acc_prin_payment,paidmoney,
 pa.status,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt,   
  case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=acc_sum) or CloseDate<=ScheduleToDate 
   then N'ontime' else N'overdue' end as pmt_stt  from pawnoverduedetail pd 
   left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))  
   left join  (select pa1.*,shop.name shopname,shop.areaid from Pawn pa1 
   left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid  
   left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid   
where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt))  
and pa.STATUS not in (15,11,140) and pa.CategoryCode like N'%17%' and InterestMoney like N'%goc%' 
and pd.scheduletodate<=@schedule_to and pd.scheduletodate>@schedule_from) tab
  left join (select pawnid pawn_wid,CodeNo,Body,CommentType,cmtdate,CreatedDate cmt_dt from CommentContract 
  where PGD is not null and cmtdate>= dateadd(day,-6,@schedule_from) and cmtdate<=@schedule_to)cmt 
  on cmt.pawn_wid=tab.pawnid and cmt.cmtdate>= dateadd(day,-6,tab.scheduletodate) and cmt.cmtdate<= tab.ScheduleToDate
 where dpd_yesterday<=5 
 order by tab.PawnID"""
    remind_stats = pd.read_sql_query(remind_str,db)
    remind_stats = remind_stats.merge(write_off_list, left_on='codeno', right_on='pawn_code',how = 'left')
    remind_stats = remind_stats[remind_stats['pawn_code'].isnull()==True]
#     remind_stats = remind_stats.merge(shop_data, left_on='shopname', right_on = 'Tên PGD', how = 'left')
#     remind_stats['khu_vuc'] = np.where(remind_stats['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(remind_stats['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((remind_stats['Tỉnh/ Thành phố'] != 'Hà Nội') & (remind_stats['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((remind_stats['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (remind_stats['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN', 'Miền Trung'))))
    need_remind = remind_stats.groupby('shopname').agg({'pawnid': 'nunique'}).reset_index()
    remind_early = remind_stats.pivot_table(index = 'shopname', columns='cmt_type', values='pawnid', aggfunc='nunique').reset_index()
    remind_early = remind_early.merge(need_remind, on = 'shopname', how='left')
    remind_early = remind_early.fillna(0)
    remind_early['early_rate_remind'] = remind_early['2.early_remind']/remind_early['pawnid']
    remind_early['ondue_rate_remind'] = remind_early['3.on_due_date']/remind_early['pawnid']
    remind_early['extract_date'] = todate
    return remind_early

In [70]:
remind_stats = remind_rate(list_date_valid[0][0],list_date_valid[0][1])

In [71]:
motor_title_early_remind = remind_stats

In [72]:
motor_title_early_remind.columns = ['shopname', 'motor_title_no_cmt', 'motor_title_early_remind', 'motor_title_on_due_date', 'motor_title_pawnid',
       'motor_title_early_rate_remind', 'motor_title_ondue_rate_remind','extract_date']

In [73]:
motor_title_early_remind = motor_title_early_remind[['extract_date','shopname','motor_title_early_rate_remind']]

In [74]:
car_title_early_remind = remind_stats_cartitle
car_title_early_remind.columns = ['shopname', 'car_title_no_cmt', 'car_title_early_remind', 'car_title_on_due_date', 'car_title_pawnid',
       'car_title_early_rate_remind', 'car_title_ondue_rate_remind', 'extract_date']

car_title_early_remind = car_title_early_remind[['extract_date','shopname','car_title_early_rate_remind']]

CASHCOUNT CHECK

In [75]:
def cash_count(fromdate, todate):
    cashcount_str = """Declare @fromdate date = '"""+fromdate+"""'
Declare @todate date = '"""+todate+"""'
select cas2.id, cas.cashcount_date, 
 case when cas.shop = 'SGN0120.220 Lý Thường Kiệt' then 'SGN0147.409C Lý Thường Kiệt'
when cas.shop = 'F88 HCM - 404 Nguyễn Tri Phương' then 'SGN0126.48 Trần Nhân Tôn'
when cas.shop = 'SGN0107.54H Nguyễn Ảnh Thủ 2' then 'F88 HCM - 54H Nguyễn Ảnh Thủ'
else cas.shop end shopname,
 cas2.net, cas2.moneyred, cas2.[DIFF] cash_diff,
case when cas2.[DIFF] < 10000 and cas2.[DIFF]> -10000 then 'f'
else 't' end diff_tf, case when cas2.NOTE_ADD is null then cas2.not_minus else cas2.NOTE_ADD end note_add,
sd.opendate, case when sd.opendate>= @fromdate and sd.opendate<=@todate then 'open_within_m'
else 'open_pre_months' end shop_open_old_new
from (select cast(created as date) cashcount_date,shop, max(ID) max_id
from [dwh].[dbo].[CASH_AMOUNT_SHOP]
where cast(CREATED as date) > @fromdate
and cast(CREATED as date) <= @todate
group by cast(created as date), shop) cas 
left join [dwh].[dbo].[CASH_AMOUNT_SHOP] cas2 on cas.max_id = cas2.id
left join [dwh].[dbo].[ShopDetail] sd on sd.name = cas.shop
where sd.code not like '%TEST%'
and sd.code not like '%HS%'
and sd.code not like '%HNIXLN01%'
and sd.code not like '%SGNXLN01%'"""
    cashcount_raw = pd.read_sql_query(cashcount_str, db)
#     cashcount_raw = cashcount_raw.merge(shop_data[['PGD', 'Mã PGD', 'GĐM', 'TPK', 'QLKV','Miền', 'Tỉnh/ Thành phố']], left_on = 'shopname', right_on = 'PGD', how = 'left')
    return cashcount_raw

In [76]:
def cashcount_check(fromdate, todate):
    cashcount_data = cash_count(fromdate, todate)
    cashcount_data['reason_length']=cashcount_data.note_add.apply(lambda x: len(str(x).split(" ")))
    cashcount_data.not_minus=cashcount_data.note_add.str.replace('[^\w\s]','')
    sub_tab_cashcount = cashcount_data[cashcount_data['note_add'].isnull() == False]
    sub_tab_cashcount.loc[(sub_tab_cashcount.note_add.str.contains('nhầm|sai|quên|mượn|thối|nhập|mất|trộm|sự cố|lỗi'))|(sub_tab_cashcount.reason_length>=30),'sub_diff_reason_check']='diff_no_clear_reason'
    sub_tab_cashcount.loc[sub_tab_cashcount.note_add.str.contains('nguyên nhân|không rõ|k rõ|tìm ra|không tìm|chưa tìm|chưa rõ'),'sub_diff_reason_check']='diff_not_found_reason' 
    sub_tab_cashcount['sub_diff_reason_check'] = np.where(sub_tab_cashcount['sub_diff_reason_check'].isnull() == True, 'diff_clear_reason', sub_tab_cashcount['sub_diff_reason_check'])
    cashcount_data = cashcount_data.merge(sub_tab_cashcount[['id', 'sub_diff_reason_check']], on = 'id', how='left')
    cashcount_data['reason_check'] = np.where(cashcount_data.diff_tf=='f', 'no_diff', np.where((cashcount_data.diff_tf =='t') & (cashcount_data.note_add.isnull()==True), 'diff_no_reason', cashcount_data['sub_diff_reason_check']))
    cashcount_data.columns = ['id', 'cashcount_date', 'shop', 'net', 'moneyred', 'cash_diff',
       'diff_tf', 'note_add', 'opendate', 'shop_open_old_new', 'reason_length',
       'sub_diff_reason_check', 'reason_check']
    count_cashcount_m_pgd = cashcount_data.pivot_table(index = ['shop_open_old_new','shop'], columns='reason_check', values='cashcount_date', aggfunc='nunique')
    count_cashcount_m_pgd_diff_rate = cashcount_data.groupby(['shop']).agg({'net':'sum','moneyred':'sum','cash_diff':'sum'}).reset_index()
    count_cashcount_m_pgd_diff_rate['cashcount_diff_rate'] = np.abs(count_cashcount_m_pgd_diff_rate['cash_diff'])/count_cashcount_m_pgd_diff_rate['net']
    count_cashcount_m_pgd = count_cashcount_m_pgd.merge(count_cashcount_m_pgd_diff_rate, on = 'shop', how = 'left')
    count_cashcount_m_pgd = count_cashcount_m_pgd.fillna(0)
    count_cashcount_m_pgd['all_cashcount'] = count_cashcount_m_pgd['diff_clear_reason']+count_cashcount_m_pgd['diff_no_clear_reason']+count_cashcount_m_pgd['diff_no_reason']+count_cashcount_m_pgd['diff_not_found_reason']+count_cashcount_m_pgd['no_diff']
    count_cashcount_m_pgd['extract_date'] = todate
    return count_cashcount_m_pgd

In [77]:
count_cashcount_m = cashcount_check(list_date_valid[0][0], list_date_valid[0][1])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
D:\Anaconda3\li

In [78]:
def cash_average_shop(fromdate, todate):
    cash_average = cash_count(fromdate, todate)
    cash_average['extract_date'] = todate
    cash_avg_summ = cash_average.groupby(['shopname', 'extract_date']).agg({'moneyred':'sum'})
    cash_avg_summ = cash_avg_summ.reset_index()
    cash_avg_summ['day_for_divide'] = [i.day for i in pd.to_datetime(cash_avg_summ['extract_date'])]
    cash_avg_summ['average_cash'] =  cash_avg_summ['moneyred']/cash_avg_summ['day_for_divide']
    return cash_avg_summ

In [79]:
average_cash_shop = cash_average_shop(list_date_valid[0][0], list_date_valid[0][1])

In [80]:
def cash_median(fromdate, todate):
    raw = cash_count(fromdate, todate)
    raw['extract_date'] = todate
    cash_median_shop = raw.groupby(['extract_date','shopname'])['moneyred'].median()
    cash_median_shop = cash_median_shop.reset_index()
    cash_median_shop = cash_median_shop.rename(columns={"moneyred": "median_cash"})
    return cash_median_shop

In [81]:
all_cash_median = cash_median(list_date_valid[0][0], list_date_valid[0][1])

In [82]:
count_cashcount_m = count_cashcount_m.merge(average_cash_shop, left_on = ['shop', 'extract_date'], right_on = ['shopname', 'extract_date'], how = 'left')

In [83]:
count_cashcount_m = count_cashcount_m.merge(all_cash_median, left_on = ['shop', 'extract_date'], right_on = ['shopname', 'extract_date'], how = 'left')

In [84]:
count_cashcount_m = count_cashcount_m[['extract_date','shop', 'diff_clear_reason', 'diff_no_clear_reason', 'diff_no_reason',
       'diff_not_found_reason', 'no_diff', 'net', 'moneyred_x', 'cash_diff',
       'cashcount_diff_rate', 'all_cashcount','average_cash', 'median_cash']]

In [85]:
cashcount_check = count_cashcount_m

DISTANCE TO HO

In [86]:
import math
from math import radians, cos, sin, sqrt, atan2

In [87]:
ho_hn_lat = radians(20.993118)
ho_hn_long = radians(105.807532)
ho_hcm_lat = radians(10.765941)
ho_hcm_long = radians(106.659512)

In [88]:
shop_detail = shop_infor

In [89]:
shop_detail['shop_lat'] = [radians(i) for i in shop_detail['Vĩ độ']]
shop_detail['shop_long'] =[radians(i) for i in shop_detail['Kinh độ']]

In [90]:
shop_detail['ho_lat'] = np.where((shop_detail['Vùng/Miền'] =='Miền Bắc')|(shop_detail['Vùng/Miền'] =='Miền Trung'), ho_hn_lat, ho_hcm_lat)
shop_detail['ho_long'] = np.where((shop_detail['Vùng/Miền'] =='Miền Bắc')|(shop_detail['Vùng/Miền'] =='Miền Trung'), ho_hn_long, ho_hcm_long)

In [91]:
def distance_to_ho(shop_lat, shop_long, ho_lat, ho_long):
    R = 6373.0
    dlon = shop_long - ho_long
    dlat = shop_lat - ho_lat

    a = sin(dlat / 2)**2 + cos(ho_lat) * cos(shop_lat) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [92]:
distances_km = []
for row in shop_detail.itertuples(index=False):
    distances_km.append(distance_to_ho(row.ho_lat, row.ho_long, row.shop_lat, row.shop_long))

In [93]:
shop_detail['distance_to_ho'] = distances_km

In [94]:
distance_to_ho = shop_detail

In [95]:
distance_to_ho = distance_to_ho[['Mã PGD', 'distance_to_ho']]

SHOP FAULT

In [96]:
# Lấy ID fault_group
fault_group_id='1X7m2UnrVh909OKk9YOkxcIB455GPwTpAAcWGHIBn6_c'
fault_group_wb=gc.open_by_key(fault_group_id)

In [97]:
fault_group_data = to_dataframe(fault_group_wb, 'pos_fault_group',1,0)

In [98]:
def shop_fault(fromdate, todate):
    shop_fault_data_str = """Declare @fromdate date = '"""+fromdate+"""'
 Declare @todate date = '"""+todate+"""'
 select cast(created as date) fault_created_date,SHOP_NAME_COMMENT, pawn_wid, contract_no,CATEGORY_CODE, status, PAWN_PROCESS_COMMENT_ERROR_CODE, COMMENT_ERROR_CODE
 from [dwh].[dbo].[W_KSNB_COMMENTS_CONTRACT_ERROR_F]
 where cast(CREATED as date) >=@fromdate
 and cast(CREATED as date) <=@todate"""
    shop_fault_data = pd.read_sql_query(shop_fault_data_str, db)
    shop_fault_data = shop_fault_data.merge(fault_group_data, left_on = 'pawn_process_comment_error_code', right_on = 'fault_code', how = 'left')
    shop_group_fault_data = shop_fault_data[shop_fault_data['status_x'] != 'Thu hồi lỗi']
    shop_fault_group = shop_group_fault_data.pivot_table(index='shop_name_comment', columns='risk',values= 'pawn_wid', aggfunc='count')
    shop_fault_group = shop_fault_group.reset_index()
    shop_fault_group = shop_fault_group.fillna(0)
    shop_fault_group['extract_date'] = todate
    return shop_fault_group

In [99]:
shop_group_fault_data = shop_fault(list_date_valid[0][0],list_date_valid[0][1])

In [100]:
shop_fault_columns = ['shop_name_comment', 'high', 'low', 'medium','significant' ,'extract_date']

In [101]:
for i in shop_fault_columns:
    if i not in shop_group_fault_data.columns:
        shop_group_fault_data[i] = np.nan

In [102]:
shop_group_fault_data.columns

Index(['shop_name_comment', 'high', 'low', 'medium', 'extract_date',
       'significant'],
      dtype='object', name='risk')

In [103]:
shop_group_fault_data = shop_group_fault_data[['extract_date','shop_name_comment','high', 'low', 'medium', 'significant']] 

In [104]:
shop_fault = shop_group_fault_data

POL

In [105]:
def pol_data(fromdate, todate):
    pol_data_str = """Declare @fromdate date ='"""+fromdate+"""' 
Declare @todate date = '"""+todate+"""'
select pawn_online_wid,DATEDIFF(minute,to_shop_date,shop_first_process_date) time_1,
DATEDIFF(minute,to_shop_date,shop_fst_dt) time_2
from [dwh].[dbo].W_pawn_online_f
where  cast(created_pol as date) >='fromdate'
and cast(created_pol as date) <='todate' 
and cast(to_shop_date as date)<= 'todate'"""

In [106]:
def pol_data(fromdate, todate):
    pol_data_str = """Declare @fromdate date ='"""+fromdate+"""' 
Declare @todate date = '"""+todate+"""'
select pol.PAWN_ONLINE_WID pol_panw, case when replace(shop_gr.shop_code, ' ','') = 'SGN0120' then 'SGN0147'
when replace(shop_gr.shop_code, ' ','') = 'SG2046' then 'SGN0126'
when replace(shop_gr.shop_code, ' ','') = 'SGN0107' then 'SG2014'
else replace(shop_gr.shop_code, ' ','') end shop_code, sd.name shopname,
case when pol.ASSET like '%ký%' then pol.ASSET else 'Khác' end asset_type,
current_groupid, steeped_groupid,
cast(to_shop_date as date) to_shop_date, cast(shop_first_process_date as date) shop_first_process_date,
shop_lst_status_str, shop_lst_content, qualified,
pol.STATUS, status_str, cancel_type, last_process_content,
created_pol, toshopdate_immediacy,
pol.pawn_online_wid, asset/*, customer_name, [url], source, campaign, datediff(minute, to_shop_date,shop_fst_dt) waiting_time_to_process,
case when  datediff(minute, to_shop_date,shop_fst_dt) <=15 then 1 else 0 end shop_process_within_15_mins,
case when  datediff(minute, to_shop_date,shop_fst_dt) <=30 then 1 else 0 end shop_process_within_30_mins*/
from [dwh].[dbo].W_pawn_online_f pol
left join [dwh].[dbo].W_group_shop_d shop_gr
left join [dwh].[dbo].ShopDetail sd on sd.code = shop_gr.SHOP_CODE
on shop_gr.group_wid=pol.current_groupid
where current_groupid <>44
and shop_gr.shop_code not like '%TEST%'
and shop_gr.shop_code not like '%HS%'
and shop_gr.shop_code not like '%HNIXLN01%'
and shop_gr.shop_code not like '%SGNXLN01%'
 and cast(created_pol as date) >=@fromdate
and cast(created_pol as date) <=@todate 
and cast(to_shop_date as date)<= @todate"""
    pol_data_raw = pd.read_sql_query(pol_data_str, db)
#     pol_data_raw = pol_data_raw.merge(shop_data[['Mã PGD', 'GĐM', 'TPK', 'QLKV',  'Miền', 'Tỉnh/ Thành phố', 'khu_vuc']], left_on = 'shop_code', right_on = 'Mã PGD', how = 'left')
    pol_data_raw['to_date'] = todate
    return pol_data_raw

In [107]:
def pol_group(fromdate, todate):
    pol_raw_data = pol_data(fromdate, todate)
    pol_convert_group = pol_raw_data.pivot_table(columns = 'shop_lst_status_str', index = ['asset_type','shop_code'], values = 'pol_panw', aggfunc = 'nunique')
    pol_convert_group = pol_convert_group.reset_index() 
    pol_convert_group = pol_convert_group.fillna(0)
    pol_all_summ = pol_raw_data.groupby(['asset_type','shop_code']).agg({'pol_panw':'nunique'})
#     pol_all_summ = pol_raw_data.groupby(['asset_type','shop_code']).agg({'pol_panw':'nunique', 'shop_process_within_15_mins':'sum', 'shop_process_within_30_mins':'sum'})
    pol_all_summ = pol_all_summ.reset_index()
    pol_convert_group = pol_convert_group.merge(pol_all_summ, on = ['asset_type','shop_code'], how = 'left')
    pol_convert_group['extract_date'] = todate
    return pol_convert_group

In [108]:
pawn_online_convert = pol_group(list_date_valid[0][0], list_date_valid[0][1])

In [109]:
pawn_online_convert = pawn_online_convert.fillna(0)

In [110]:
pawn_online_convert['success_rate'] = pawn_online_convert['Nhận Cầm Cố']/pawn_online_convert['pol_panw']
pawn_online_convert['cancelled_rate'] = pawn_online_convert['Hủy đăng ký']/pawn_online_convert['pol_panw']

In [111]:
pawn_online_convert = pawn_online_convert.drop(columns='')

In [112]:
pawn_online_convert_summ = pawn_online_convert[['extract_date', 'shop_code','asset_type', 'Hủy đăng ký', 'Nhận Cầm Cố', 'PGD đang chăm sóc','pol_panw', 'success_rate', 'cancelled_rate']]

In [113]:
pawn_online_convert_summ.columns = ['extract_date', 'shop_code','asset_type','cancelled', 'success_convert', 'taking_care', 'all', 'success_rate', 'cancelled_rate']
pol_cartitle = pawn_online_convert_summ[pawn_online_convert_summ['asset_type'] == 'Đăng ký Ô tô']

pol_motortitle = pawn_online_convert_summ[pawn_online_convert_summ['asset_type'] == 'Đăng ký xe máy']

pol_normalasset = pawn_online_convert_summ[pawn_online_convert_summ['asset_type'] == 'Khác']

In [114]:
pol_cartitle.columns = ['extract_date', 'shop_code', 'asset_type', 'cartt_cancelled', 'cartt_success_convert',
       'cartt_taking_care', 'cartt_all', 'cartt_success_rate', 'cartt_cancelled_rate']

In [115]:
pol_motortitle.columns = ['extract_date', 'shop_code', 'asset_type',
       'motortt_cancelled', 'motortt_success_convert',
       'motortt_taking_care', 'motortt_all', 'motortt_success_rate', 'motortt_cancelled_rate',]

In [116]:
pol_normalasset.columns = ['extract_date', 'shop_code', 'asset_type',
       'normass_cancelled', 'normass_success_convert',
       'normass_taking_care', 'normass_all', 'normass_success_rate', 'normass_cancelled_rate']

In [371]:
type(pol_cartitle['extract_date'][527])

pandas._libs.tslibs.timestamps.Timestamp

TRAFFIC

In [117]:
def traffic_data(fromdate, todate):
    traff_raw_str = """Declare @fromdate date ='"""+fromdate+"""' 
Declare @todate date = '"""+todate+"""'
select * from
(select  shop.AreaID areaid, 
case when shop.code = 'SGN0120' then N'SGN0147.409C Lý Thường Kiệt'
when shop.code = 'SG2046' then N'SGN0126.48 Trần Nhân Tôn'
when shop.code = 'SGN0107' then N'F88 HCM - 54H Nguyễn Ảnh Thủ'
else shop.name end shopname,
shop.OpenDate open_date,
tra.TRAFFIC_NUM traffic_num, tra.CTR_OPEN_NUM contract_open,
case when cate.NAME like '%ký%' then cate.NAME else 'others' end categoryname
from 
(
select CATEGORY_WID, SHOP_WID, sum(traffic_num) TRAFFIC_NUM, sum(ctr_open_num)  CTR_OPEN_NUM
from [dwh].[dbo].[W_TRAFFIC_LOAN_DAILY_F] 
where  CREATED> @fromdate
and CREATED<= @todate
group by CATEGORY_WID, SHOP_WID) tra
left join [dwh].[dbo].shopdetail shop on shop.shopid=tra.SHOP_WID
left join [dwh].[dbo].W_CATEGORY_D cate on tra.category_wid=cate.CATEGORY_WID
where shop.code not like '%TEST%'
and shop.code not like '%HS%'
and shop.code not like '%HNIXLN01%'
and shop.code not like '%SGNXLN01%') main
where main.contract_open <= main.traffic_num"""
    traff_raw = pd.read_sql_query(traff_raw_str, db)
    traff_raw['convert_rate'] = traff_raw['contract_open']/traff_raw['traffic_num']
    traff_raw['extract_date'] = todate
    return traff_raw

In [118]:
traffic_data_raw = traffic_data(list_date_valid[0][0],list_date_valid[0][1])

In [119]:
traffic_data_raw_group = traffic_data_raw.groupby(['categoryname','shopname', 'extract_date']).agg({'traffic_num':'sum', 'contract_open':'sum'})

In [120]:
traffic_data_raw_group = traffic_data_raw_group.reset_index()

In [121]:
traffic_data_raw_group['convert_rate'] = traffic_data_raw_group['contract_open']/traffic_data_raw_group['traffic_num']

In [122]:
cartitle_traffic_rate = traffic_data_raw_group[traffic_data_raw_group['categoryname'] == 'Đăng ký Ô tô']

motortitle_traffic_rate = traffic_data_raw_group[traffic_data_raw_group['categoryname'] == 'Đăng ký xe máy']

norm_asset_traffic_rate = traffic_data_raw_group[traffic_data_raw_group['categoryname'] == 'others']

In [123]:
cartitle_traffic_rate = cartitle_traffic_rate[[ 'shopname','categoryname','traffic_num','contract_open','convert_rate', 'extract_date']]
motortitle_traffic_rate = motortitle_traffic_rate[[ 'shopname','categoryname','traffic_num','contract_open','convert_rate', 'extract_date']]
norm_asset_traffic_rate = norm_asset_traffic_rate[[ 'shopname','categoryname','traffic_num','contract_open','convert_rate', 'extract_date']]

In [124]:
cartitle_traffic_rate.columns = ['shopname','categoryname', 'cartt_traffic','cartt_contract_open','cartt_convert_rate','extract_date']
motortitle_traffic_rate.columns = ['shopname','categoryname', 'motortt_traffic','motortt_contract_open','motortt_convert_rate','extract_date']
norm_asset_traffic_rate.columns = ['shopname','categoryname', 'normass_traffic','normass_contract_open','normass_convert_rate','extract_date']

In [125]:
cartitle_traffic_rate = cartitle_traffic_rate[['extract_date','shopname', 'cartt_traffic','cartt_contract_open', 'cartt_convert_rate']]

motortitle_traffic_rate = motortitle_traffic_rate[['extract_date','shopname', 'motortt_traffic', 'motortt_contract_open', 'motortt_convert_rate']]

norm_asset_traffic_rate = norm_asset_traffic_rate[['extract_date','shopname', 'normass_traffic', 'normass_contract_open', 'normass_convert_rate']]

AUDIT SCHEDULE

In [1161]:
# Lấy ID audit schedule
audit_schedule='1tzePinmzoopkaiq5NQ_wch3N_o5W5G0eC8qrjPYLviI'
audit_schedule_wb=client.open_by_key(audit_schedule)

In [1162]:
audit_schedule=audit_schedule_wb.worksheet_by_title('schedule').get_as_df()

In [1163]:
audit_schedule['audit_month'] = [i.month for i in pd.to_datetime(audit_schedule['audit_date'])]

In [1164]:
audit_schedule['thang'] = np.where(audit_schedule['audit_month'] == datetime.strptime(running_check, "%Y-%m-%d").month, audit_schedule['audit_month'], audit_schedule['audit_month'] + 1)

In [1165]:
audit_schedule_maxtime = audit_schedule.pivot_table(index='shopname', columns='audit_type', values='audit_date',aggfunc='max' ).reset_index()

In [1166]:
shop_infor.columns

Index(['shopcode', 'shopname', 'opendate', 'Mã PGD', 'Tỉnh/TP', 'Vĩ độ',
       'Kinh độ', 'Vùng/Miền', 'key', 'extract_date', 'yearmonth',
       'province_id', 'province_name', 'province_risk_level'],
      dtype='object')

In [1167]:
shop_infor['extract_date'] = pd.to_datetime(shop_infor['extract_date'], format="%Y-%m-%d")
shop_infor['opendate'] = pd.to_datetime(shop_infor['opendate'], format="%Y-%m-%d")
shop_infor['based_date'] = pd.to_datetime('2020-04-30', format="%Y-%m-%d")

In [1168]:
shop_infor['final_date'] = np.where(shop_infor['opendate']>shop_infor['based_date'], shop_infor['opendate'], shop_infor['based_date'])

In [1169]:
audit_schedule_direct = audit_schedule[audit_schedule['audit_type'] == 'Direct']

audit_schedule_indirect = audit_schedule[audit_schedule['audit_type'] == 'Indirect']

In [1170]:
audit_schedule_indirect.columns

Index(['shopcode', 'shopname', 'region', 'province', 'audit_date',
       'audit_yearmonth', 'audit_type', 'Thời gian', 'auditor', 'audit_month',
       'thang'],
      dtype='object')

In [1171]:
audit_schedule_direct.columns

Index(['shopcode', 'shopname', 'region', 'province', 'audit_date',
       'audit_yearmonth', 'audit_type', 'Thời gian', 'auditor', 'audit_month',
       'thang'],
      dtype='object')

In [1172]:
shop_infor['thang'] = pd.to_datetime(shop_infor['extract_date'], format="%Y-%m-%d")

In [1173]:
shop_infor['thang'] = [i.month for i in shop_infor['thang']]

In [1174]:
shop_infor_direct = shop_infor.merge(audit_schedule_direct, on = ['shopname', 'thang'], how = 'left')

In [1175]:
shop_infor_indirect = shop_infor.merge(audit_schedule_indirect, on = ['shopname', 'thang'], how = 'left')

In [1176]:
shop_infor_direct = shop_infor_direct[['shopname','opendate','extract_date','audit_date','final_date']]

In [1177]:
shop_infor_direct.columns = ['shopname','opendate','extract_date','direct_audit_date','final_date']

In [1178]:
shop_infor_indirect = shop_infor_indirect[['shopname','opendate','extract_date','audit_date','final_date']]
shop_infor_indirect.columns = ['shopname','opendate','extract_date','indirect_audit_date','final_date']

In [1179]:
shop_infor_direct = shop_infor_direct.merge(audit_schedule_maxtime, on = 'shopname', how = 'left')

In [1180]:
shop_infor_direct['final_date'] =pd.to_datetime(shop_infor_direct['final_date'], format='%Y-%m-%d') 

In [1181]:
shop_infor_direct = shop_infor_direct.drop(columns='Indirect')

shop_infor_direct['Direct'] =pd.to_datetime(shop_infor_direct['Direct'], format='%Y-%m-%d') 

In [1182]:
shop_infor_direct['date_for_diff'] = np.where(shop_infor_direct['direct_audit_date'].isnull()==False,shop_infor_direct['direct_audit_date'],shop_infor_direct['Direct'])

shop_infor_direct['date_for_diff'] = np.where(shop_infor_direct['date_for_diff'].isnull()==False, shop_infor_direct['date_for_diff'], shop_infor_direct['final_date'])

shop_infor_direct['date_for_diff'] =pd.to_datetime(shop_infor_direct['date_for_diff'], format='%Y-%m-%d')

In [1183]:
shop_infor_direct['sub_date_for_diff'] = np.where(shop_infor_direct['date_for_diff']<= shop_infor_direct['extract_date'], shop_infor_direct['date_for_diff'], shop_infor_direct['direct_audit_date'])

shop_infor_direct['sub_date_for_diff'] = np.where(shop_infor_direct['sub_date_for_diff'].isnull() == False,shop_infor_direct['sub_date_for_diff'], shop_infor_direct['final_date'])

shop_infor_direct['sub_date_for_diff'] =pd.to_datetime(shop_infor_direct['sub_date_for_diff'], format='%Y-%m-%d') 

shop_infor_direct['nums_diff_day_from_last_direct_audit'] = shop_infor_direct['extract_date'] - shop_infor_direct['sub_date_for_diff']

shop_infor_direct['nums_diff_day_from_last_direct_audit'] = pd.to_numeric(shop_infor_direct['nums_diff_day_from_last_direct_audit'].dt.days, downcast='integer')

shop_infor_direct = shop_infor_direct[shop_infor_direct['nums_diff_day_from_last_direct_audit']>0]

In [1184]:
shop_infor_indirect = shop_infor_indirect.merge(audit_schedule_maxtime, on = 'shopname', how = 'left')

In [1185]:
shop_infor_indirect = shop_infor_indirect.drop(columns='Direct')

In [1187]:
shop_infor_indirect['Indirect'] =pd.to_datetime(shop_infor_indirect['Indirect'], format='%Y-%m-%d') 

In [1188]:
shop_infor_indirect['date_for_diff'] = np.where(shop_infor_indirect['indirect_audit_date'].isnull()==False,shop_infor_indirect['indirect_audit_date'],shop_infor_indirect['Indirect'])

shop_infor_indirect['date_for_diff'] = np.where(shop_infor_indirect['date_for_diff'].isnull()==False, shop_infor_indirect['date_for_diff'], shop_infor_indirect['final_date'])

shop_infor_indirect['date_for_diff'] =pd.to_datetime(shop_infor_indirect['date_for_diff'], format='%Y-%m-%d')

In [1189]:
shop_infor_indirect['sub_date_for_diff'] = np.where(shop_infor_indirect['date_for_diff']<= shop_infor_indirect['extract_date'], shop_infor_indirect['date_for_diff'], shop_infor_indirect['indirect_audit_date'])

shop_infor_indirect['sub_date_for_diff'] = np.where(shop_infor_indirect['sub_date_for_diff'].isnull() == False,shop_infor_indirect['sub_date_for_diff'], shop_infor_indirect['final_date'])

shop_infor_indirect['sub_date_for_diff'] =pd.to_datetime(shop_infor_indirect['sub_date_for_diff'], format='%Y-%m-%d') 

In [1200]:
shop_infor_indirect['nums_diff_day_from_last_indirect_audit'] = pd.to_numeric(shop_infor_indirect['nums_diff_day_from_last_indirect_audit'].dt.days, downcast='integer')

In [1202]:
shop_infor_indirect = shop_infor_indirect[shop_infor_indirect['nums_diff_day_from_last_direct_audit']>=0]

In [1203]:
shop_infor_direct = shop_infor_direct[['shopname', 'extract_date','nums_diff_day_from_last_direct_audit']]

In [1204]:
shop_infor_indirect = shop_infor_indirect[['shopname', 'extract_date','nums_diff_day_from_last_indirect_audit']]

In [1205]:
shop_infor_direct['extract_date'] = pd.to_datetime(shop_infor_direct['extract_date'],format='%Y-%m-%d')
shop_infor_indirect['extract_date'] = pd.to_datetime(shop_infor_indirect['extract_date'],format='%Y-%m-%d')

In [1206]:
direct_audit_schedule = shop_infor_direct
indirect_audit_schedule = shop_infor_indirect

In [1207]:
direct_audit_schedule = direct_audit_schedule[['extract_date', 'shopname', 'nums_diff_day_from_last_direct_audit']]

In [1208]:
indirect_audit_schedule= indirect_audit_schedule[['extract_date','shopname','nums_diff_day_from_last_indirect_audit']]

25% Quartile

In [164]:
def open_period(fromdate, todate):
    open_period_str = """Declare @fromdate date = '"""+fromdate+"""'
Declare @todate date = '"""+todate+"""'
select le.EXTRAC_DATE,support_staff,created_staff,ShopCode,le.CodeNo,Customer,case when categorycode like '%17%' then 'DKXM'
when  categorycode like '%15%' then 'DKOT'
else 'SPTT' end categoryname,OpenHour,InitTime,complete, datediff(minute,OpenHour, complete) open_period
from OPEN_CLOSE_CONTRACT_BY_TIME le 
left join W_CATEGORY_D cate on cate.CODE=le.CategoryCode 
left join (select codeno,FirstSupportID,u.USER_CODE support_staff,u2.USER_CODE created_staff,ShopCode,InitTime from pawn pa1 
left join W_USER_F u on pa1.firstsupportid=u.USER_WID 
left join W_USER_F u2 on pa1.CreatedBy=u2.USER_WID )pa on le.CodeNo=pa.CodeNo 
left join (select PAWN_WID,CONTRACT_NO,min(CREATED)complete 
from W_PAWN_TRANSACTION_F where action_name like N'Cho vay' group by PAWN_WID,CONTRACT_NO)trans on trans.CONTRACT_NO=le.CodeNo 
where OpenHour is not null 
and cast (OpenHour as date) > @fromdate
and cast (OpenHour as date) <= @todate"""
    open_period = pd.read_sql_query(open_period_str, db)
    open_period['extract_date'] = todate
    return open_period

In [165]:
def one_forth_quartile_norm(fromdate, todate):
    data_raw = open_period(fromdate, todate)
    normal_asset_quartile_open_period = data_raw[data_raw['categoryname']=='SPTT'].groupby('shopcode')[['open_period']].describe().reset_index()
    normal_asset_quartile_open_period.columns = ['_'.join(col) for col in normal_asset_quartile_open_period.columns.values]
    normal_asset_quartile_open_period = normal_asset_quartile_open_period[['shopcode_', 'open_period_25%']]
    normal_asset_quartile_open_period['extract_date'] = todate
    normal_asset_quartile_open_period.columns = ['shopcode', 'normal_asset_25%', 'extract_date']
    return normal_asset_quartile_open_period

In [166]:
def one_forth_quartile_motor_title(fromdate, todate):
    data_raw = open_period(fromdate, todate)
    motortt_quartile_open_period = data_raw[data_raw['categoryname']=='DKXM'].groupby('shopcode')[['open_period']].describe().reset_index()
    motortt_quartile_open_period.columns = ['_'.join(col) for col in motortt_quartile_open_period.columns.values]
    motortt_quartile_open_period = motortt_quartile_open_period[['shopcode_', 'open_period_25%']]
    motortt_quartile_open_period['extract_date'] = todate
    motortt_quartile_open_period.columns = ['shopcode', 'motor_title_25%', 'extract_date']
    return motortt_quartile_open_period

In [167]:
def one_forth_quartile_car_title(fromdate, todate):
    data_raw = open_period(fromdate, todate)
    quartile_open_period = data_raw[data_raw['categoryname']=='DKOT'].groupby('shopcode')[['open_period']].describe().reset_index()
    quartile_open_period.columns = ['_'.join(col) for col in quartile_open_period.columns.values]
    quartile_open_period = quartile_open_period[['shopcode_', 'open_period_25%']]
    quartile_open_period['extract_date'] = todate
    quartile_open_period.columns = ['shopcode', 'car_title_25%', 'extract_date']
    return quartile_open_period

In [168]:
motor_title_all = one_forth_quartile_motor_title(list_date_valid[0][0],list_date_valid[0][1])
car_title_all = one_forth_quartile_car_title(list_date_valid[0][0],list_date_valid[0][1])
normal_asset_all = one_forth_quartile_norm(list_date_valid[0][0],list_date_valid[0][1])

In [169]:
all_one_forth = motor_title_all.merge(car_title_all, on = ['shopcode', 'extract_date']).merge(normal_asset_all, on = ['shopcode', 'extract_date'])

In [171]:
motor_title_one_forth_open_time = motor_title_all

car_title_one_forth_open_time = car_title_all
normal_asset_one_forth_open_time = normal_asset_all

In [172]:
motor_title_one_forth_open_time['extract_date'] = pd.to_datetime(motor_title_one_forth_open_time['extract_date'], format="%Y-%m-%d")
car_title_one_forth_open_time['extract_date'] = pd.to_datetime(car_title_one_forth_open_time['extract_date'], format="%Y-%m-%d")
normal_asset_one_forth_open_time['extract_date'] = pd.to_datetime(normal_asset_one_forth_open_time['extract_date'], format="%Y-%m-%d")

DISBUR PAWN DAY

In [173]:
def disbur_pawn_day(fromdate, todate):
    disbur_pawn_day_str = """Declare @fromdate date = '"""+fromdate+"""'
Declare @todate date = '"""+todate+"""'
  select fromdate, shopcode, pawnid, case when CategoryCode like '%15%' then 'DKOT'
  when CategoryCode like '%17%' then 'DKXM'
  else 'SPTT' end category_name
FROM [dwh].[dbo].[Pawn]
  where fromdate>@fromdate
  and fromdate<=@todate
  and ShopCode not like '%HS%'
  and ShopCode not like '%TEST%'
  and status not in (11,15,22)"""
    disbur_pawn_day = pd.read_sql_query(disbur_pawn_day_str, db)
    return disbur_pawn_day

In [174]:
def disbur_group(fromdate, todate):
    trial = disbur_pawn_day(fromdate, todate)
    trial['extract_date'] = todate
    disbur_pawn = trial.pivot_table(index = ['extract_date','shopcode'], columns='category_name', values='pawnid', aggfunc='nunique').reset_index()
    disbur_pawn.columns = ['extract_date','shopcode', 'nums_disbur_pawn_DKOT', 'nums_disbur_pawn_DKXM', 'nums_disbur_pawn_SPTT']
    disbur_day = trial.pivot_table(index = ['extract_date','shopcode'], columns='category_name', values='fromdate', aggfunc='nunique').reset_index()
    disbur_day.columns = ['extract_date','shopcode', 'nums_disbur_days_DKOT', 'nums_disbur_days_DKXM', 'nums_disbur_days_SPTT']
    disbur_group_pawn_days = disbur_day.merge(disbur_pawn, on = ['extract_date','shopcode'], how = 'left')
    disbur_group_pawn_days = disbur_group_pawn_days.fillna(0)
    disbur_group_pawn_days['avg_disbur_pawn_car_title'] = disbur_group_pawn_days['nums_disbur_pawn_DKOT']/ disbur_group_pawn_days['nums_disbur_days_DKOT']
    disbur_group_pawn_days['avg_disbur_pawn_motor_title'] = disbur_group_pawn_days['nums_disbur_pawn_DKXM']/ disbur_group_pawn_days['nums_disbur_days_DKXM']
    disbur_group_pawn_days['avg_disbur_pawn_normal_asset'] = disbur_group_pawn_days['nums_disbur_pawn_SPTT']/ disbur_group_pawn_days['nums_disbur_days_SPTT']
    return disbur_group_pawn_days

In [175]:
disbur_pawn_days = disbur_group(list_date_valid[0][0],list_date_valid[0][1]) 

In [176]:
disbur_pawn_day = disbur_pawn_days

In [177]:
disbur_pawn_day = disbur_pawn_day[['extract_date', 'shopcode', 'nums_disbur_days_DKOT',
       'nums_disbur_days_DKXM', 'nums_disbur_days_SPTT','avg_disbur_pawn_car_title',
       'avg_disbur_pawn_motor_title', 'avg_disbur_pawn_normal_asset']]

In [178]:
disbur_pawn_day['extract_date'] = pd.to_datetime(disbur_pawn_day['extract_date'], format="%Y-%m-%d")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


KPI

In [197]:
write_off_id='15r30_u_xE0iUexkUgsbZqcZTy0htmSflCp-uty_1hWA'
write_off_wb=gc.open_by_key(write_off_id)
write_off=to_dataframe(write_off_wb,'list',1,0)

write_off.amount=write_off.amount.astype(int)

write_off.write_off_period=write_off.write_off_period.astype(int)

# Lấy danh sách write-off
write_off['code']=write_off.codeno.apply(lambda x: x[4:])

In [182]:
shop_detail_id='1V34xp4Dl4ROUEX_jiG-AiwPUspemhJ0MfxUz8Tvkkzg'
shop_detail_wb=gc.open_by_key(shop_detail_id)

In [183]:
shop_control=to_dataframe(shop_detail_wb,'Quản lý KD',1,0)

In [184]:
kpi_data=to_dataframe(shop_detail_wb,'KPI for Master Report',1,0)

kpi_data['KPI Dư nợ']=kpi_data['KPI Dư nợ'].apply(lambda x: int(x.replace(',','')))

kpi_data=kpi_data[kpi_data.PGD.str.contains('Hội Sở')==False]

kpi_proces=kpi_data.groupby(['Năm KPI','Tháng KPI','PGD','Tỉnh/TP','Miền','TPK','ASM'],as_index=False).agg({'KPI Online lead':sum,'KPI Online Sale':sum,'KPI Traffic':sum,'KPI Dư nợ':sum})

In [207]:
def read_bal_kpi(ext_date,kpi_month):
    bal_str="declare @ext_date date='"+ext_date+"'select created,yearmonth,ContractCode,SUBSTRING(contractcode,5,len(contractcode)) as code,CustomerName,CategoryName,FromDate,ToDate,currentMoney,ShopName,ShopCode,ShopID,TuoiNo from pawnoverdue  where categoryname not like N'%vị%' and (shopcode not like N'%TEST%' or shopcode not like N'%HS%') and created=@ext_date"
    bal_data=pd.read_sql_query(bal_str,db)
    bal_merge=pd.merge(bal_data,write_off[['write_off_period','code']],on='code',how='left')
    bal_merge.loc[bal_merge.write_off_period.isna(),'write_off_period']=9999999
    bal_kpi=bal_merge[(bal_merge.write_off_period>bal_merge.yearmonth.max())&(bal_merge.tuoino<11)]
    bal_kpi_group=bal_kpi.groupby(['created','shopid','shopcode','shopname'],as_index=False).agg({'currentmoney':sum})
    bal_kpi_f=pd.merge(bal_kpi_group,kpi_proces[kpi_proces['Tháng KPI']==kpi_month],left_on='shopname',right_on='PGD',how='left')
    bal_kpi_f['bal_gap']=bal_kpi_f['KPI Dư nợ']-bal_kpi_f.currentmoney
    bal_kpi_f['complete_rate']=bal_kpi_f.currentmoney/bal_kpi_f['KPI Dư nợ']
    bal_kpi_f['extract_date'] = ext_date
    return bal_kpi_f

In [204]:
date_list_kpi = [[check_date, str(datetime.strptime(check_date, "%Y-%m-%d").month)]]

In [205]:
date_list_kpi

[['2020-11-10', '11']]

In [208]:
bal_kpi=read_bal_kpi(date_list_kpi[0][0],date_list_kpi[0][1])

In [209]:
kpi = bal_kpi

In [210]:
kpi = kpi[['extract_date','shopcode', 'complete_rate']]

In [211]:
kpi = kpi.rename(columns={'complete_rate':'kpi_complete_rate'})

In [212]:
kpi['extract_date'] = pd.to_datetime(kpi['extract_date'], format="%Y-%m-%d")

YIELD

In [214]:
#Lấy ID và truy cập wb Backup
yield_id='1wMtWeZOwqEwPNLWBwsjKInzXlcGhzjEgzFLAER6yyIg'
yield_wb=client.open_by_key(yield_id)

yield_result_id='13SOYuJ3iJEESgNElzIY0glovNLoaDkjKtRFSFC-C1g4'
yield_result_wb=client.open_by_key(yield_result_id)

#  Lấy ID shopdetail
shop_detail_id='1ZTQE_pfBCAUr-0GSMPJBqtCyZVxX4hdiPuT_RDrdjeQ'
shop_detail_wb=client.open_by_key(shop_detail_id)

shopdetail_data=shop_detail_wb.worksheet_by_title('shop_detail').get_as_df()

shop_detail=shopdetail_data[['Mã PGD','PGD','Miền','Tỉnh/ Thành phố','Tháng KT','QLKV','TPK']]
shop_detail.columns=['shopcode','shopname','area','province_name','open_month','asm','rsm']
shop_detail['province_type']=shop_detail.province_name
shop_detail.loc[(shop_detail.province_name!='Hà Nội')&(shop_detail.area=='Miền Bắc'),'province_type']='Tỉnh.MB'
shop_detail.loc[(shop_detail.province_name!='TP. Hồ Chí Minh')&(shop_detail.area=='Miền Nam'),'province_type']='Tỉnh.MN'
shop_detail.loc[shop_detail.area=='Miền Trung','province_type']='Miền Trung'

def to_df(wb_name,sheet_name):
    return wb_name.worksheet_by_title(sheet_name).get_as_df()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1

In [215]:
received_int=to_df(yield_wb,'511')
received_fees=to_df(yield_wb,'711')
write_off=to_df(yield_wb,'writeoff')
provision=to_df(yield_wb,'provision')
net_balance=to_df(yield_wb,'netbalance')
inventory=to_df(yield_wb,'inventory')

In [216]:
def cate_yield(cate_name):
    received_int_group=received_int[received_int.cate.isin(cate_name)].groupby(['ym','shopcode'],as_index=False).agg({'int':sum})
    received_fees_group=received_fees[received_fees.cate.isin(cate_name)].groupby(['ym','shopcode'],as_index=False).agg({'fees':sum})
    write_off_group=write_off[write_off.cate.isin(cate_name)].groupby(['ym','shopcode'],as_index=False).agg({'writeoff_amt':sum})
    provision_group=provision[provision.cate.isin(cate_name)].groupby(['ym','shopcode'],as_index=False).agg({'provided_amt':sum})
    net_balance_group=net_balance[net_balance.cate.isin(cate_name)].groupby(['ym','shopcode'],as_index=False).agg({'net_bal':sum})
    inventory_group=inventory[inventory.cate.isin(cate_name)].groupby(['ym','shopcode'],as_index=False).agg({'inventory_amt':sum})
    return received_int_group,received_fees_group,write_off_group,provision_group,net_balance_group,inventory_group
    

In [217]:
def yield_cal(ym_result,cate_list):
    
    received_int_group,received_fees_group,write_off_group,provision_group,net_balance_group,inventory_group=cate_yield(cate_list)
    
    # Khởi tạo bảng kết quả
    result=net_balance_group.loc[net_balance_group.ym==(ym_result-1),['shopcode','net_bal']]
    result.columns=['shopcode','ob_net_bal']
    cb_net_balance=net_balance_group.loc[net_balance_group.ym==ym_result,['shopcode','net_bal']]
    cb_net_balance.columns=['shopcode','cb_net_bal']
    pre2_net_balance=net_balance_group.loc[net_balance_group.ym==(ym_result-2),['shopcode','net_bal']]
    pre2_net_balance.columns=['shopcode','pre_2_month_net_balance']
    
    # Write-off phát sinh tháng T-1
    pre_month_wo=write_off_group.loc[write_off_group.ym==(ym_result-1),['shopcode','writeoff_amt']]
    pre_month_wo.columns=['shopcode','pre_month_wo']
    result=pd.merge(result,pre2_net_balance,on='shopcode',how='left')
    result=pd.merge(result,cb_net_balance,on='shopcode',how='left')
    result=pd.merge(result,pre_month_wo,on='shopcode',how='left')
    ##### Tính Mẫu số

    # Writeoff luỹ kế đến hết tháng T-2
    acc_writeoff=write_off_group[(write_off_group.ym!=201912)&(write_off_group.ym<=(ym_result-2))].groupby('shopcode',as_index=False).agg({'writeoff_amt':sum})
    acc_writeoff.columns=['shopcode','acc_wo_pre_2_months']
    
    # Provision đầu kỳ
    ob_provision=provision_group.loc[provision_group.ym==(ym_result-1),['shopcode','provided_amt']]
    ob_provision.columns=['shopcode','ob_provision']

    # HTK đầu kỳ
    ob_inventory=inventory_group.loc[inventory_group.ym==(ym_result-1),['shopcode','inventory_amt']]
    ob_inventory.columns=['shopcode','ob_inventory']

    result=pd.merge(result,acc_writeoff,on='shopcode',how='left')
    result=pd.merge(result,ob_provision,on='shopcode',how='left')
    result=pd.merge(result,ob_inventory,on='shopcode',how='left')
    

    ##### Tính tử số

    # Tính 511
    cb_received_int=received_int_group.loc[received_int_group.ym==ym_result,['shopcode','int']]
    cb_received_int.columns=['shopcode','actual_int_received']
    # Tính 711
    cb_received_fees=received_fees_group.loc[received_fees_group.ym==ym_result,['shopcode','fees']]
    cb_received_fees.columns=['shopcode','actual_fee_received']
    # Writeoff current
    cb_writeoff=write_off_group.loc[write_off_group.ym==ym_result,['shopcode','writeoff_amt']]
    cb_writeoff.columns=['shopcode','wo_incurred']
    # Provision current
    cb_provision=provision_group.loc[provision_group.ym==ym_result,['shopcode','provided_amt']]
    cb_provision.columns=['shopcode','provision_incurred']

    result=pd.merge(result,cb_received_int,on='shopcode',how='left')
    result=pd.merge(result,cb_received_fees,on='shopcode',how='left')
    result=pd.merge(result,cb_writeoff,on='shopcode',how='left')
    result=pd.merge(result,cb_provision,on='shopcode',how='left')

    result=result.fillna(0)

    ##### Tính Yield

    # Tính Yield
    result['yield']=(result.actual_int_received+result.actual_fee_received-result.provision_incurred-result.wo_incurred)/(result.ob_net_bal+result.ob_inventory+result.acc_wo_pre_2_months-result.ob_provision)

    result.insert(0,'ym_result',ym_result)
    return result

In [ ]:
this_day_of_last_month = datetime.strptime(check_date, "%Y-%m-%d") - datetime.del

In [236]:
today = datetime.strptime(check_date, "%Y-%m-%d")
first = today.replace(day=1)
lastMonth = first - timedelta(days=1)

In [237]:
ym_result_range = [datetime.strptime(check_date, "%Y-%m-%d").strftime('%Y%m'), lastMonth.strftime("%Y%m")]

In [350]:
ym_result_range

[202011, 202010]

In [242]:
ym_result_range = [int(i) for i in ym_result_range]

In [243]:
car_cate_list=['Đăng ký Ô tô']

car_yield_data=yield_cal(ym_result_range[0],car_cate_list)


for car_ym_result in ym_result_range[1:]:
    yield_data=car_yield_data.append(yield_cal(car_ym_result,car_cate_list))
yield_data['ob_net_bal_grow']=(yield_data.ob_net_bal- yield_data.pre_2_month_net_balance)/yield_data.pre_2_month_net_balance
yield_data['tu_so']=yield_data.actual_int_received+yield_data.actual_fee_received-yield_data.wo_incurred-yield_data.provision_incurred
yield_data['mau_so']=yield_data.ob_net_bal+yield_data.ob_inventory-yield_data.ob_provision
yield_data['int/ob_net_bal']=yield_data.actual_int_received/yield_data.ob_net_bal
yield_data['wo_incurred/ob_bal']=yield_data.wo_incurred/yield_data.ob_net_bal

dkot_yield_shop=pd.merge(yield_data,shop_detail,on='shopcode',how='left')

dkot_yield_shop.loc[dkot_yield_shop.province_type.isna(),'area']='Miền Nam'
dkot_yield_shop.loc[dkot_yield_shop.province_type.isna(),'province_name']='TP. Hồ Chí Minh'
dkot_yield_shop.loc[dkot_yield_shop.province_type.isna(),'province_type']='TP. Hồ Chí Minh'

In [244]:
motor_cate_list=['Đăng ký xe máy']

motor_yield_data=yield_cal(ym_result_range[0],motor_cate_list)
for motor_ym_result in ym_result_range[1:]:
    motor_yield_data=motor_yield_data.append(yield_cal(motor_ym_result,motor_cate_list))
motor_yield_data['ob_net_bal_grow']=(motor_yield_data.ob_net_bal- motor_yield_data.pre_2_month_net_balance)/motor_yield_data.pre_2_month_net_balance
motor_yield_data['tu_so']=motor_yield_data.actual_int_received+motor_yield_data.actual_fee_received-motor_yield_data.wo_incurred-motor_yield_data.provision_incurred
motor_yield_data['mau_so']=motor_yield_data.ob_net_bal+motor_yield_data.ob_inventory-motor_yield_data.ob_provision
motor_yield_data['int/ob_net_bal']=motor_yield_data.actual_int_received/motor_yield_data.ob_net_bal
motor_yield_data['wo_incurred/ob_bal']=motor_yield_data.wo_incurred/motor_yield_data.ob_net_bal

dkxm_yield_shop=pd.merge(motor_yield_data,shop_detail,on='shopcode',how='left')

dkxm_yield_shop.loc[dkxm_yield_shop.province_type.isna(),'area']='Miền Nam'
dkxm_yield_shop.loc[dkxm_yield_shop.province_type.isna(),'province_name']='TP. Hồ Chí Minh'
dkxm_yield_shop.loc[dkxm_yield_shop.province_type.isna(),'province_type']='TP. Hồ Chí Minh'

In [245]:
yiel_index_motor_title = dkxm_yield_shop

In [246]:
yiel_index_motor_title['thang'] = pd.to_datetime(yiel_index_motor_title['ym_result'], format="%Y-%m")

In [247]:
yiel_index_motor_title['thang'] = [i.month for i in yiel_index_motor_title['thang']]

In [248]:
yiel_index_motor_title= yiel_index_motor_title[['thang','shopcode','yield']]

In [249]:
yiel_index_car_title = dkot_yield_shop

In [250]:
yiel_index_car_title['thang'] = pd.to_datetime(yiel_index_car_title['ym_result'], format="%Y-%m")
yiel_index_car_title['thang'] = [i.month for i in yiel_index_car_title['thang']]

In [251]:
yiel_index_car_title= yiel_index_car_title[['thang','shopcode','yield']]

In [252]:
yiel_index_motor_title.columns = ['thang', 'shopcode', 'yield_motor_title']
yiel_index_car_title.columns = ['thang', 'shopcode', 'yield_car_title']

LIQUID RATE

In [253]:
def liquid_asset_rate(fromdate):
    liquid_asset_str = """Declare @fromdate date = '"""+fromdate+"""'
  select pod.pawnid, pod.shopcode, pod.shopname, case when  pod.CategoryName = 'Ô tô' then 'car'
  when pod.CategoryName = 'Xe máy' then 'motorbike'
  else 'normal_asset' end asset_type,
  case when pa.status = -1 then 1 else 0 end asset_liquid
  from pawnoverdue pod
  left join pawn pa on pa.Pawnid = pod.PawnID
  where pod.CategoryCode not like '%15%'
  and pod.CategoryCode not like '%17%'
  and pod.CREATED = @fromdate
    and pod.ShopCode not like '%HS%'
  and pod.ShopCode not like '%TEST%'"""
    liquid_asset = pd.read_sql_query(liquid_asset_str, db)
    liquid_all = liquid_asset.groupby(['shopcode', 'shopname']).agg({'pawnid':'count'}).reset_index()
    liquid_by_asset_type = liquid_asset.pivot_table(index = ['shopcode', 'shopname'], columns='asset_type', values='asset_liquid', aggfunc= 'sum').reset_index()
    liquid_by_asset_type = liquid_by_asset_type.merge(liquid_all, on = ['shopcode', 'shopname'], how = 'left' )
    liquid_by_asset_type = liquid_by_asset_type.fillna(0)
    liquid_by_asset_type['liquid_car_rate'] = liquid_by_asset_type['car']/liquid_by_asset_type['pawnid']
    liquid_by_asset_type['liquid_motorbike_rate'] = liquid_by_asset_type['motorbike']/liquid_by_asset_type['pawnid']
    liquid_by_asset_type['liquid_normal_asset_rate'] = liquid_by_asset_type['normal_asset']/liquid_by_asset_type['pawnid']
    liquid_by_asset_type['extract_date'] = fromdate
    return liquid_by_asset_type

In [254]:
liquid_asset_this_month = liquid_asset_rate(list_date_valid[0][0])

In [255]:
liquid_asset_this_month.columns = ['shopcode', 'shopname', 'nums_liquid_car', 'nums_liquid_motorbike', 'nums_liquid_other_asset', 'total_normal_asset',
       'car_liquid_rate', 'motorbike_liquid_rate', 'normal_asset_liquid_rate','extract_date']

In [256]:
liquid_rate_m = liquid_asset_this_month

In [257]:
liquid_rate_m['thang'] = pd.to_datetime(liquid_rate_m['extract_date'], format="%Y-%m-%d")
liquid_rate_m['thang'] = [i.month for i in liquid_rate_m['thang']]

In [258]:
liquid_rate = liquid_rate_m[['thang','shopcode','car_liquid_rate',
       'motorbike_liquid_rate', 'normal_asset_liquid_rate']]

CLOSED PAWN

In [259]:
#  Closed_pawn
def closed_pawn_type_1(fromdate, todate):
    closed_pawn_type_1_str = """Declare @start_date date ='"""+fromdate+"""'
    Declare @end_date date ='"""+todate+"""'
select case when pod.categorycode like '%15%' then 'DKOT' 
when pod.categorycode like '%17%' then 'DKXM' else 'SPTT' end categoryname,pod.shopcode,pod.ShopName shopname,pod.contractcode, pod.pawnid pod_pawn,
cast(left(pod.Interest,4) as int) interest, pa.*,
case  when pa.Pawnid is null then 0 else 1 end real_closed_pawn
from  [dwh].[dbo].[PawnOverdue] pod
left join (select a.*, case when a.closedate < a.todate and a.closedate<=dateadd(day, 10, a.fromdate) and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '1. early closed'
when  a.closedate = a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '2. ondue closed'
when a.closedate > a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '3. overdue_closed'
 when a.closedate=a.following_fromdate then '4.close to open new'
 else '5.just closed' end "closed_type"
 from (
select [CategoryCode], pawnid, fromdate, todate, closedate,  Lead (Pawnid) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) as following_pawn_id,
 Lead (fromdate) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) as following_fromdate,
  Lead(CodeNo) over (PARTITION BY customerid, CategoryCode ORDER BY fromdate asc) as foll_pawn, 
  Lead(Pawnid) over (PARTITION BY customerid, CategoryCode ORDER BY fromdate asc) as foll_pawn_id
 from [dwh].[dbo].pawn
 where closedate> @start_date
 and closedate<=@end_date) a) pa on pa.pawnid = pod.PawnID
 left join [dwh].[dbo].pawn pa2 on pa2.pawnid = pod.pawnid
 where pod.CREATED = @start_date
 and pod.shopcode not like '%HS%' 
and pod.shopcode not like '%TEST%'
and pod.TuoiNo<90"""
    closed_type_1 = pd.read_sql_query(closed_pawn_type_1_str, db)
    closed_type_1_m_overview = closed_type_1.groupby(['categoryname','shopcode', 'shopname']).agg({'pod_pawn':'nunique','real_closed_pawn':'sum'}).reset_index()
    closed_type_1_compare_to_due = closed_type_1.pivot_table(index = ['categoryname','shopcode', 'shopname'], columns= 'closed_type', values='pod_pawn',aggfunc='nunique').reset_index()
    closed_type_1_compare_to_due = closed_type_1_compare_to_due.fillna(0)
    closed_type_1_m_overview = closed_type_1_m_overview.merge(closed_type_1_compare_to_due, on = ['categoryname','shopcode', 'shopname'], how = 'left')
    closed_type_1_m_overview['extract_date'] = todate
    return closed_type_1_m_overview

In [260]:
closed_type_1_m = closed_pawn_type_1(list_date_valid[0][0], list_date_valid[0][1])

In [261]:
closed_type_1_m = closed_type_1_m.fillna(0)

In [262]:
closed_type_1_m['closed_rate'] = closed_type_1_m['real_closed_pawn']/closed_type_1_m['pod_pawn']
closed_type_1_m['early_closed_rate'] = closed_type_1_m['1. early closed']/closed_type_1_m['pod_pawn']
closed_type_1_m['closed_to_open_new_per_total_closed'] = closed_type_1_m['4.close to open new']/closed_type_1_m['real_closed_pawn']

In [263]:
# Pawn open in this month:
def closed_pawn_type_2(fromdate,todate):
    closed_type_2_str = """Declare @start_date date ='"""+fromdate+"""'
Declare @end_date date ='"""+todate+"""'
select a.*,case when cd.code like '%15%' then 'DKOT' 
when cd.code like '%17%' then 'DKXM' else 'SPTT' end categoryname , case when closedate is null then '0.still_open'
when closedate is not null and a.closedate < a.todate and a.closedate<=dateadd(day, 10, a.fromdate) and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '1. early closed'
when closedate is not null and a.closedate = a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '2. ondue closed'
when closedate is not null and a.closedate > a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '3. overdue_closed'
 when closedate is not null and a.closedate=a.following_fromdate then '4.close to open new'
 else '5.just closed' end "closed_type"
from (
select shopcode, rate,[CategoryCode], pawnid, fromdate, todate, closedate,  LoanMoneyOrg,[InsurranceMoney],
Lead (Pawnid) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) as following_pawn_id,
 Lead (fromdate) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) as following_fromdate
 from [dwh].[dbo].pawn
 where 1=1
 and fromdate> @start_date
 and fromdate<=@end_date
 and shopcode not like '%HS%' and 
shopcode not like '%TEST%'
) a
 left join [dwh].[dbo].W_CATEGORY_D cd on cd.CODE = a.CategoryCode"""
    closed_type_2 = pd.read_sql_query(closed_type_2_str, db)
    closed_type_2_m_overview = closed_type_2.pivot_table(index = ['categoryname','shopcode'], columns='closed_type', values='pawnid', aggfunc='nunique').reset_index()
    closed_type_2_m_overview = closed_type_2_m_overview.fillna(0)
    closed_type_2_m_overview['extract_date'] = todate
    return closed_type_2_m_overview

In [264]:
closed_type_2_m = closed_pawn_type_2(list_date_valid[0][0],list_date_valid[0][1])

In [265]:
closed_type_2_m = closed_type_2_m.fillna(0)

In [266]:
closed_type_2_m['all'] = closed_type_2_m['0.still_open']+closed_type_2_m['1. early closed']+closed_type_2_m['4.close to open new']
closed_type_2_m['total_closed'] = closed_type_2_m['1. early closed']+closed_type_2_m['4.close to open new']
closed_type_2_m['closed_rate'] = closed_type_2_m['total_closed']/closed_type_2_m['all']
closed_type_2_m['early_closed_rate'] = closed_type_2_m['1. early closed']/closed_type_2_m['all']
closed_type_2_m['closed_to_open_new_rate'] = closed_type_2_m['4.close to open new']/closed_type_2_m['total_closed']

In [267]:
overview_closed_type_1_m = closed_type_1_m[['categoryname','extract_date','shopcode','pod_pawn', 'real_closed_pawn','1. early closed','4.close to open new']]

overview_closed_type_2_m = closed_type_2_m[['categoryname','extract_date', 'shopcode','all', 'total_closed','1. early closed','4.close to open new']]

In [268]:
overview_closed_pawn = overview_closed_type_1_m.merge(overview_closed_type_2_m, on = ['categoryname','extract_date','shopcode'], how = 'left')

In [269]:
overview_closed_pawn['total'] = overview_closed_pawn['pod_pawn']+overview_closed_pawn['all']
overview_closed_pawn['all_closed'] = overview_closed_pawn['real_closed_pawn']+overview_closed_pawn['total_closed']
overview_closed_pawn['all_early_closed'] = overview_closed_pawn['1. early closed_x']+overview_closed_pawn['1. early closed_y']
overview_closed_pawn['all_closed_to_open_new'] = overview_closed_pawn['4.close to open new_x']+overview_closed_pawn['4.close to open new_y']

overview_closed_pawn['closed_rate'] = overview_closed_pawn['all_closed']/overview_closed_pawn['total']
overview_closed_pawn['early_closed_rate'] = overview_closed_pawn['all_early_closed']/overview_closed_pawn['total']
overview_closed_pawn['closed_to_open_new_rate'] = overview_closed_pawn['all_closed_to_open_new']/overview_closed_pawn['all_closed']

overview_closed_pawn = overview_closed_pawn[['categoryname','extract_date','shopcode', 'closed_rate','early_closed_rate', 'closed_to_open_new_rate']]

In [271]:
closed_pawn = overview_closed_pawn

In [272]:
closed_pawn['extract_date'] = pd.to_datetime(closed_pawn['extract_date'], format="%Y-%m-%d")

In [273]:
closed_pawn_car_title = closed_pawn[closed_pawn['categoryname'] == 'DKOT']
closed_pawn_motor_title = closed_pawn[closed_pawn['categoryname'] == 'DKXM']
closed_pawn_normal_asset = closed_pawn[closed_pawn['categoryname'] == 'SPTT']

In [274]:
closed_pawn_car_title.columns = ['categoryname', 'extract_date', 'shopcode', 'car_title_closed_rate',
       'car_title_early_closed_rate', 'car_title_closed_to_open_new_rate']
closed_pawn_motor_title.columns = ['categoryname', 'extract_date', 'shopcode', 'motor_title_closed_rate',
       'motor_title_early_closed_rate', 'motor_title_closed_to_open_new_rate']
closed_pawn_normal_asset.columns = ['categoryname', 'extract_date', 'shopcode', 'normal_asset_closed_rate',
       'normal_asset_early_closed_rate', 'normal_asset_closed_to_open_new_rate']

In [275]:
closed_pawn_car_title = closed_pawn_car_title[['extract_date', 'shopcode', 'car_title_closed_rate',
       'car_title_early_closed_rate', 'car_title_closed_to_open_new_rate']]
closed_pawn_motor_title = closed_pawn_motor_title[['extract_date', 'shopcode', 'motor_title_closed_rate',
       'motor_title_early_closed_rate', 'motor_title_closed_to_open_new_rate']]
closed_pawn_normal_asset = closed_pawn_normal_asset[['extract_date', 'shopcode', 'normal_asset_closed_rate',
       'normal_asset_early_closed_rate', 'normal_asset_closed_to_open_new_rate']]

SHOP AGE MONTH

In [1005]:
asm_risk_lv = to_dataframe(fault_group_wb, 'asm_risk_level',1,0)

In [1006]:
shop_age_month_str = """
 select code shopcode, month(getdate()) thang, DATEDIFF(month, OpenDate, GETDATE()) shop_age_m
 FROM [dwh].[dbo].[ShopDetail]
union all
( select code shopcode, month(dateadd(m,-1,getdate())) thang, DATEDIFF(month, OpenDate, dateadd(m,-1,getdate())) shop_age_m
 FROM [dwh].[dbo].[ShopDetail])
 union all
 (select code shopcode, month(dateadd(m,-2,getdate())) thang, DATEDIFF(month, OpenDate, dateadd(m,-2,getdate())) shop_age_m
 FROM [dwh].[dbo].[ShopDetail])
 union all
 (select code shopcode, month(dateadd(m,-3,getdate())) thang, DATEDIFF(month, OpenDate, dateadd(m,-3,getdate())) shop_age_m
 FROM [dwh].[dbo].[ShopDetail])
"""

In [1007]:
shop_age_month = pd.read_sql_query(shop_age_month_str, db)

In [1008]:
shop_detail_mkt.columns

Index(['Mã PGD', 'QLKV'], dtype='object')

In [1009]:
shop_age_month = shop_age_month.merge(shop_detail_mkt, left_on = 'shopcode', right_on = 'Mã PGD', how = 'left')

In [1010]:
staff_tenure_str = """
select month(getdate()) thang, shopcode, sum(tenure) as staff_tenure
from
(SELECT 
user1.UserFullName, user1.UserName, user1.UserEmail, user1.UserID, 
case when group1.code = 'ch_k1112_cvh' then 'HNK11VH'
when group1.code = 'ch_381_cg' then 'HN381CG'
WHEN group1.code = '218DC' THEN 'HN218DC'
WHEN group1.code = 'HNDR' THEN 'DR'
WHEN group1.code = 'HN982L' THEN '850DL'
WHEN group1.code = '218DC' THEN 'HN218DC'
else group1.code end as shopcode, user1.created, datediff(month, user1.created, getdate()) as tenure
FROM [dwh].[dbo].[User] user1
left join [dwh].[dbo].[W_USER_GROUP_D] usergroup on usergroup.user_id = user1.userid
left join [dwh].[dbo].[W_GROUP_D] group1 on group1.group_wid = usergroup.group_id
where user1.STATUS = 1 and group1.isshop = 1 and user1.useremail like '%@f88.co%' and user1.userposition = 4) staff
group by shopcode
union all 
(select month(dateadd(month, -1, getdate())) thang, shopcode, sum(tenure) as staff_tenure
from
(SELECT 
user1.UserFullName, user1.UserName, user1.UserEmail, user1.UserID, 
case when group1.code = 'ch_k1112_cvh' then 'HNK11VH'
when group1.code = 'ch_381_cg' then 'HN381CG'
WHEN group1.code = '218DC' THEN 'HN218DC'
WHEN group1.code = 'HNDR' THEN 'DR'
WHEN group1.code = 'HN982L' THEN '850DL'
WHEN group1.code = '218DC' THEN 'HN218DC'
else group1.code end as shopcode, user1.created, datediff(month, user1.created, dateadd(month, -1, getdate())) as tenure
FROM [dwh].[dbo].[User] user1
left join [dwh].[dbo].[W_USER_GROUP_D] usergroup on usergroup.user_id = user1.userid
left join [dwh].[dbo].[W_GROUP_D] group1 on group1.group_wid = usergroup.group_id
where user1.STATUS = 1 and group1.isshop = 1 and user1.useremail like '%@f88.co%' and user1.userposition = 4) staff
group by shopcode)
union all 
(select month(dateadd(month, -2, getdate())) thang, shopcode, sum(tenure) as staff_tenure
from
(SELECT 
user1.UserFullName, user1.UserName, user1.UserEmail, user1.UserID, 
case when group1.code = 'ch_k1112_cvh' then 'HNK11VH'
when group1.code = 'ch_381_cg' then 'HN381CG'
WHEN group1.code = '218DC' THEN 'HN218DC'
WHEN group1.code = 'HNDR' THEN 'DR'
WHEN group1.code = 'HN982L' THEN '850DL'
WHEN group1.code = '218DC' THEN 'HN218DC'
else group1.code end as shopcode, user1.created, datediff(month, user1.created, dateadd(month, -2, getdate())) as tenure
FROM [dwh].[dbo].[User] user1
left join [dwh].[dbo].[W_USER_GROUP_D] usergroup on usergroup.user_id = user1.userid
left join [dwh].[dbo].[W_GROUP_D] group1 on group1.group_wid = usergroup.group_id
where user1.STATUS = 1 and group1.isshop = 1 and user1.useremail like '%@f88.co%' and user1.userposition = 4) staff
group by shopcode)
union all 
(select month(dateadd(month, -3, getdate())) thang, shopcode, sum(tenure) as staff_tenure
from
(SELECT 
user1.UserFullName, user1.UserName, user1.UserEmail, user1.UserID, 
case when group1.code = 'ch_k1112_cvh' then 'HNK11VH'
when group1.code = 'ch_381_cg' then 'HN381CG'
WHEN group1.code = '218DC' THEN 'HN218DC'
WHEN group1.code = 'HNDR' THEN 'DR'
WHEN group1.code = 'HN982L' THEN '850DL'
WHEN group1.code = '218DC' THEN 'HN218DC'
else group1.code end as shopcode, user1.created, datediff(month, user1.created, dateadd(month, -3, getdate())) as tenure
FROM [dwh].[dbo].[User] user1
left join [dwh].[dbo].[W_USER_GROUP_D] usergroup on usergroup.user_id = user1.userid
left join [dwh].[dbo].[W_GROUP_D] group1 on group1.group_wid = usergroup.group_id
where user1.STATUS = 1 and group1.isshop = 1 and user1.useremail like '%@f88.co%' and user1.userposition = 4) staff
group by shopcode)"""

In [1011]:
staff_tenure = pd.read_sql_query(staff_tenure_str, db)

In [1012]:
manager_tenure_str = """
select month(getdate()) thang, shopcode, sum(tenure) as manager_tenure
from
(SELECT 
user1.UserFullName, user1.UserName, user1.UserEmail, user1.UserID, 
case when group1.code = 'ch_k1112_cvh' then 'HNK11VH'
when group1.code = 'ch_381_cg' then 'HN381CG'
WHEN group1.code = '218DC' THEN 'HN218DC'
WHEN group1.code = 'HNDR' THEN 'DR'
WHEN group1.code = 'HN982L' THEN '850DL'
WHEN group1.code = '218DC' THEN 'HN218DC'
else group1.code end as shopcode, user1.created, datediff(month, user1.created, getdate()) as tenure
FROM [dwh].[dbo].[User] user1
left join [dwh].[dbo].[W_USER_GROUP_D] usergroup on usergroup.user_id = user1.userid
left join [dwh].[dbo].[W_GROUP_D] group1 on group1.group_wid = usergroup.group_id
where user1.STATUS = 1 and group1.isshop = 1 and user1.useremail like '%@f88.co%' and user1.userposition = 3) staff
group by shopcode
union all 
(select month(dateadd(month, -1, getdate())) thang, shopcode, sum(tenure) as manager_tenure
from
(SELECT 
user1.UserFullName, user1.UserName, user1.UserEmail, user1.UserID, 
case when group1.code = 'ch_k1112_cvh' then 'HNK11VH'
when group1.code = 'ch_381_cg' then 'HN381CG'
WHEN group1.code = '218DC' THEN 'HN218DC'
WHEN group1.code = 'HNDR' THEN 'DR'
WHEN group1.code = 'HN982L' THEN '850DL'
WHEN group1.code = '218DC' THEN 'HN218DC'
else group1.code end as shopcode, user1.created, datediff(month, user1.created, dateadd(month, -1, getdate())) as tenure
FROM [dwh].[dbo].[User] user1
left join [dwh].[dbo].[W_USER_GROUP_D] usergroup on usergroup.user_id = user1.userid
left join [dwh].[dbo].[W_GROUP_D] group1 on group1.group_wid = usergroup.group_id
where user1.STATUS = 1 and group1.isshop = 1 and user1.useremail like '%@f88.co%' and user1.userposition = 3) staff
group by shopcode)
union all 
(select month(dateadd(month, -2, getdate())) thang, shopcode, sum(tenure) as manager_tenure
from
(SELECT 
user1.UserFullName, user1.UserName, user1.UserEmail, user1.UserID, 
case when group1.code = 'ch_k1112_cvh' then 'HNK11VH'
when group1.code = 'ch_381_cg' then 'HN381CG'
WHEN group1.code = '218DC' THEN 'HN218DC'
WHEN group1.code = 'HNDR' THEN 'DR'
WHEN group1.code = 'HN982L' THEN '850DL'
WHEN group1.code = '218DC' THEN 'HN218DC'
else group1.code end as shopcode, user1.created, datediff(month, user1.created, dateadd(month, -2, getdate())) as tenure
FROM [dwh].[dbo].[User] user1
left join [dwh].[dbo].[W_USER_GROUP_D] usergroup on usergroup.user_id = user1.userid
left join [dwh].[dbo].[W_GROUP_D] group1 on group1.group_wid = usergroup.group_id
where user1.STATUS = 1 and group1.isshop = 1 and user1.useremail like '%@f88.co%' and user1.userposition = 3) staff
group by shopcode)
union all 
(select month(dateadd(month, -3, getdate())) thang, shopcode, sum(tenure) as manager_tenure
from
(SELECT 
user1.UserFullName, user1.UserName, user1.UserEmail, user1.UserID, 
case when group1.code = 'ch_k1112_cvh' then 'HNK11VH'
when group1.code = 'ch_381_cg' then 'HN381CG'
WHEN group1.code = '218DC' THEN 'HN218DC'
WHEN group1.code = 'HNDR' THEN 'DR'
WHEN group1.code = 'HN982L' THEN '850DL'
WHEN group1.code = '218DC' THEN 'HN218DC'
else group1.code end as shopcode, user1.created, datediff(month, user1.created, dateadd(month, -3, getdate())) as tenure
FROM [dwh].[dbo].[User] user1
left join [dwh].[dbo].[W_USER_GROUP_D] usergroup on usergroup.user_id = user1.userid
left join [dwh].[dbo].[W_GROUP_D] group1 on group1.group_wid = usergroup.group_id
where user1.STATUS = 1 and group1.isshop = 1 and user1.useremail like '%@f88.co%' and user1.userposition = 3) staff
group by shopcode)"""

In [1013]:
manager_tenure = pd.read_sql_query(manager_tenure_str, db)

In [1014]:
asm_tenure_str = """select month(getdate()) thang, userfullname, datediff(month,created, getdate()) asm_tenure 
FROM [dwh].[dbo].[User]
where UserPosition = 2
union all
(select month(dateadd(month, -1, getdate())) thang, userfullname, datediff(month,created, dateadd(month, -1, getdate())) asm_tenure 
FROM [dwh].[dbo].[User]
where UserPosition = 2)
union all
(select month(dateadd(month, -2, getdate())) thang, userfullname, datediff(month,created, dateadd(month, -2, getdate())) asm_tenure 
FROM [dwh].[dbo].[User]
where UserPosition = 2)
union all
(select month(dateadd(month, -3, getdate())) thang, userfullname, datediff(month,created, dateadd(month, -3, getdate())) asm_tenure 
FROM [dwh].[dbo].[User]
where UserPosition = 2)"""

In [1015]:
asm_tenure = pd.read_sql_query(asm_tenure_str,db)

In [1016]:
manager_tenure.columns

Index(['thang', 'shopcode', 'manager_tenure'], dtype='object')

In [1017]:
shop_age_month = shop_age_month.merge(asm_tenure, left_on= ['thang', 'shopcode'], right_on=['thang', 'userfullname'], how = 'left')

In [1018]:
shop_age_month = shop_age_month.merge(staff_tenure, on= ['thang', 'shopcode'], how = 'left')

In [1019]:
shop_age_month = shop_age_month.merge(manager_tenure, on= ['thang', 'shopcode'], how = 'left')

In [1020]:
shop_age_month = shop_age_month[['shopcode', 'thang', 'shop_age_m', 'staff_tenure', 'manager_tenure','asm_tenure']]

shop_age_month = shop_age_month[shop_age_month['shop_age_m'].isnull() == False]

In [1021]:
shop_age_month

,shopcode,thang,shop_age_m,staff_tenure,manager_tenure,asm_tenure
8,BG,11,64.0,18.0,25.0,NaN
9,SG2050,11,14.0,6.0,18.0,NaN
10,SG2051,11,14.0,49.0,13.0,NaN
11,SG2052,11,14.0,10.0,15.0,NaN
12,HN505NH,11,14.0,23.0,28.0,NaN
13,SGN0101,11,14.0,35.0,14.0,NaN
14,SGN0102,11,14.0,19.0,18.0,NaN
15,BDG0103,11,14.0,14.0,18.0,NaN
16,DNI0232,11,2.0,6.0,10.0,NaN
17,HPG0233,11,2.0,6.0,17.0,NaN


TEMPLATE TABLE

In [562]:
shop_fault['extract_date'] = pd.to_datetime(shop_fault['extract_date'], format="%Y-%m-%d")

In [563]:
rolled_bo['extract_date'] = pd.to_datetime(rolled_bo['extract_date'], format="%Y-%m-%d")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [564]:
balance_ratio_cartitle['extract_date'] = pd.to_datetime(balance_ratio_cartitle['extract_date'], format="%Y-%m-%d")
balance_ratio_motortitle['extract_date'] = pd.to_datetime(balance_ratio_motortitle['extract_date'], format="%Y-%m-%d")

In [565]:
growth_rate['extract_date'] = pd.to_datetime(growth_rate['extract_date'], format="%Y-%m-%d")

In [566]:
ontime_motortiltle_rate['extract_date'] = pd.to_datetime(ontime_motortiltle_rate['extract_date'], format="%Y-%m-%d")
fav['extract_date'] = pd.to_datetime(fav['extract_date'], format="%Y-%m-%d")
ontime_cartitle_rate['extract_date'] = pd.to_datetime(ontime_cartitle_rate['extract_date'], format="%Y-%m-%d")
motor_title_early_remind['extract_date'] = pd.to_datetime(motor_title_early_remind['extract_date'], format="%Y-%m-%d")
car_title_early_remind['extract_date'] = pd.to_datetime(car_title_early_remind['extract_date'], format="%Y-%m-%d")
cashcount_check['extract_date'] = pd.to_datetime(cashcount_check['extract_date'], format="%Y-%m-%d")
balance_ratio_cartitle_pawn['extract_date'] = pd.to_datetime(balance_ratio_cartitle_pawn['extract_date'], format="%Y-%m-%d")
balance_ratio_motortitle_pawn['extract_date'] = pd.to_datetime(balance_ratio_motortitle_pawn['extract_date'], format="%Y-%m-%d")
balance_ratio_normal_asset['extract_date'] = pd.to_datetime(balance_ratio_normal_asset['extract_date'], format="%Y-%m-%d")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [567]:
pol_cartitle['extract_date'] = pd.to_datetime(pol_cartitle['extract_date'], format="%Y-%m-%d")
cartitle_traffic_rate['extract_date'] = pd.to_datetime(cartitle_traffic_rate['extract_date'], format="%Y-%m-%d")
motortitle_traffic_rate['extract_date'] = pd.to_datetime(motortitle_traffic_rate['extract_date'], format="%Y-%m-%d")
norm_asset_traffic_rate['extract_date'] = pd.to_datetime(norm_asset_traffic_rate['extract_date'], format="%Y-%m-%d")
direct_audit_schedule['extract_date'] = pd.to_datetime(direct_audit_schedule['extract_date'], format="%Y-%m-%d")
indirect_audit_schedule['extract_date'] = pd.to_datetime(indirect_audit_schedule['extract_date'], format="%Y-%m-%d")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [568]:
pol_motortitle['extract_date'] = pd.to_datetime(pol_motortitle['extract_date'], format="%Y-%m-%d")
pol_normalasset['extract_date'] = pd.to_datetime(pol_normalasset['extract_date'], format="%Y-%m-%d")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [800]:
template_table_data.columns

Index(['shopcode', 'shopname', 'opendate', 'Mã PGD_x', 'Tỉnh/TP', 'Vĩ độ',
       'Kinh độ', 'Vùng/Miền', 'key', 'extract_date', 'yearmonth',
       'province_id', 'province_name', 'province_risk_level', 'Mã PGD_y',
       'distance_to_ho', 'shop_name_comment', 'high', 'low', 'medium',
       'significant', 'rolled_rate_motor_title', 'rolled_rate_car_title',
       'thang', 'growth_rate_car_title', 'growth_rate_motor_title',
       'growth_rate_normal_asset', 'nums_grownet_cartitle',
       'nums_grownet_motortitle', 'nums_grownet_normalasset',
       'total_nums_growthnet'],
      dtype='object')

In [1224]:
template_table_data = shop_infor.merge(distance_to_ho, left_on = 'shopcode', right_on = 'Mã PGD',how = 'left')

In [1225]:
template_table_data['extract_date'] = pd.to_datetime(template_table_data['extract_date'], format="%Y-%m-%d")

In [1226]:
template_table_data = template_table_data.merge(shop_fault, left_on = ['extract_date','shopname'], right_on = ['extract_date','shop_name_comment'],how = 'left')

In [1227]:
template_table_data['shopcode'].count()

240

In [1228]:
template_table_data = template_table_data.merge(rolled_bo, on = ['extract_date','shopcode'], how ='left')

In [1229]:
template_table_data = template_table_data.merge(growth_rate, on = ['extract_date','shopcode'], how = 'left')

In [1230]:
template_table_data = template_table_data.merge(balance_ratio_cartitle, on = ['extract_date','shopcode'], how = 'left')

In [1231]:
template_table_data.columns

Index(['shopcode', 'shopname', 'opendate', 'Mã PGD_x', 'Tỉnh/TP', 'Vĩ độ',
       'Kinh độ', 'Vùng/Miền', 'key', 'extract_date', 'yearmonth',
       'province_id', 'province_name', 'province_risk_level', 'Mã PGD_y',
       'distance_to_ho', 'shop_name_comment', 'high', 'low', 'medium',
       'significant', 'rolled_rate_motor_title', 'rolled_rate_car_title',
       'thang', 'growth_rate_car_title', 'growth_rate_motor_title',
       'growth_rate_normal_asset', 'nums_grownet_cartitle',
       'nums_grownet_motortitle', 'nums_grownet_normalasset',
       'total_nums_growthnet', '0.cartitle_dpd<=0_rate',
       '1.cartitle_0<dpd<10_rate', '2.cartitle_10<=dpd<90_rate',
       '3.cartitle_dpd>=90_rate', 'cartt_0.dpd<11'],
      dtype='object')

In [1232]:
template_table_data = template_table_data.merge(balance_ratio_motortitle, on = ['extract_date','shopcode'], how = 'left')

In [1233]:
template_table_data = template_table_data.merge(balance_ratio_normal_asset, on = ['extract_date','shopcode'], how = 'left')

In [1234]:
ontime_motortiltle_rate = ontime_motortiltle_rate[['extract_date','shopname', 'motortitle_ontime_rate']]

In [1235]:
template_table_data = template_table_data.merge(ontime_motortiltle_rate, on = ['extract_date','shopname'], how = 'left')

In [1236]:
template_table_data = template_table_data.merge(ontime_cartitle_rate, on = ['extract_date','shopname'], how = 'left')

In [1237]:
template_table_data = template_table_data.merge(motor_title_early_remind, on = ['extract_date','shopname'], how = 'left')

In [1238]:
template_table_data = template_table_data.merge(car_title_early_remind, on = ['extract_date','shopname'], how = 'left')

In [1239]:
template_table_data = template_table_data.merge(cashcount_check, left_on = ['extract_date','shopname'], right_on = ['extract_date','shop'], how = 'left')

In [1240]:
template_table_data = template_table_data.merge(balance_ratio_cartitle_pawn, on = ['extract_date','shopcode'], how = 'left')

In [1241]:
template_table_data = template_table_data.merge(balance_ratio_motortitle_pawn, on = ['extract_date','shopcode'], how = 'left')

In [1242]:
# template_table_data = template_table_data.merge(balance_ratio_normal_asset, on = ['extract_date','shopcode'], how = 'left')

In [1243]:
template_table_data = template_table_data.merge(pol_cartitle, left_on=['extract_date','shopcode'], right_on=['extract_date','shop_code'], how = 'left')

In [1244]:
template_table_data = template_table_data.merge(pol_motortitle, left_on=['extract_date','shopcode'], right_on=['extract_date','shop_code'], how = 'left')

In [1245]:
template_table_data = template_table_data.merge(pol_normalasset, left_on=['extract_date','shopcode'], right_on=['extract_date','shop_code'], how = 'left')

In [1246]:
template_table_data = template_table_data.merge(cartitle_traffic_rate, on = ['extract_date','shopname'], how = 'left')

In [1247]:
template_table_data = template_table_data.merge(motortitle_traffic_rate, on = ['extract_date','shopname'], how = 'left')

In [1248]:
template_table_data = template_table_data.merge(norm_asset_traffic_rate, on = ['extract_date','shopname'], how = 'left')

In [1249]:
template_table_data = template_table_data.fillna(0)

In [1250]:
template_table_data['all_traffic'] = template_table_data['cartt_traffic']+template_table_data['motortt_traffic']+template_table_data['normass_traffic']

In [1251]:
template_table_data['all_traffic_contract_open'] = template_table_data['cartt_contract_open']+template_table_data['motortt_contract_open']+template_table_data['normass_contract_open']

In [1252]:
template_table_data['all_traffic_convert_rate'] = template_table_data['all_traffic_contract_open']/template_table_data['all_traffic']

In [1253]:
template_table_data['total_to_shop'] = template_table_data['motortt_all']+template_table_data['cartt_all']+template_table_data['normass_all']

In [1254]:
template_table_data['total_success'] = template_table_data['cartt_success_convert']+template_table_data['motortt_success_convert']+template_table_data['normass_success_convert']

template_table_data['total_cancelled'] = template_table_data['cartt_cancelled']+template_table_data['motortt_cancelled']+template_table_data['normass_cancelled']

In [1255]:
template_table_data['rate_total_success'] = template_table_data['total_success']/template_table_data['total_to_shop']
template_table_data['rate_total_cancelled'] = template_table_data['total_cancelled']/template_table_data['total_to_shop']

In [1256]:
template_table_data['cartt_0.dpd<11'] = template_table_data['cartt_0.dpd<11'].fillna(0) 

In [1257]:
template_table_data['motortt_0.dpd<11'] = template_table_data['motortt_0.dpd<11'].fillna(0)

In [1258]:
# template_table_data.head(5).to_excel('variable_list.xlsx')

In [1259]:
template_table_data['normass_0.dpd<11'] = template_table_data['normass_0.dpd<11'].fillna(0)

In [1260]:
template_table_data['all_dpd<11_bal'] = template_table_data['cartt_0.dpd<11']+template_table_data['motortt_0.dpd<11']+template_table_data['normass_0.dpd<11']

In [1261]:
template_table_data['average_motor_title_balance_dpdlessthan11'] = template_table_data['motortt_0.dpd<11']/(template_table_data['motortt_num_pawn_0. dpd<=0']+template_table_data['motortt_num_pawn_1. 0<dpd<10'])

In [1262]:
template_table_data['average_car_title_balance_dpdlessthan11'] = template_table_data['cartt_0.dpd<11']/(template_table_data['cartt_num_pawn_0. dpd<=0']+template_table_data['cartt_num_pawn_1. 0<dpd<10'])

In [1263]:
template_table_data = template_table_data.merge(direct_audit_schedule, on = ['extract_date', 'shopname'], how = 'left')

In [1264]:
template_table_data = template_table_data.merge(indirect_audit_schedule, on = ['extract_date', 'shopname'], how = 'left')

In [1265]:
template_table_data = template_table_data.merge(motor_title_one_forth_open_time, on =['shopcode', 'extract_date'], how = 'left')

In [1266]:
template_table_data = template_table_data.merge(car_title_one_forth_open_time, on =['shopcode', 'extract_date'], how = 'left')

In [1267]:
template_table_data = template_table_data.merge(normal_asset_one_forth_open_time, on =['shopcode', 'extract_date'], how = 'left')

In [1268]:
template_table_data = template_table_data.merge(disbur_pawn_day, on = ['extract_date', 'shopcode'], how = 'left')

In [1269]:
template_table_data['nums_day'] = [i.day for i in template_table_data['extract_date']]

In [1270]:
template_table_data['car_title_nums_day_disbur_rate'] = template_table_data['nums_disbur_days_DKOT']/template_table_data['nums_day']

In [1271]:
template_table_data['motor_title_nums_day_disbur_rate'] = template_table_data['nums_disbur_days_DKXM']/template_table_data['nums_day']

In [1272]:
template_table_data['normal_asset_nums_day_disbur_rate'] = template_table_data['nums_disbur_days_SPTT']/template_table_data['nums_day']

In [1273]:
template_table_data = template_table_data[['thang','extract_date','shopcode','shopname','Tỉnh/TP',
'Vĩ độ','Kinh độ','province_id','province_risk_level','distance_to_ho','high','low','medium','significant',
'rolled_rate_motor_title','rolled_rate_car_title',
 'growth_rate_car_title','growth_rate_motor_title','growth_rate_normal_asset',
'nums_grownet_cartitle','nums_grownet_motortitle','nums_grownet_normalasset',
                                           
'0.cartitle_dpd<=0_rate','1.cartitle_0<dpd<10_rate','2.cartitle_10<=dpd<90_rate','3.cartitle_dpd>=90_rate',
'cartt_0.dpd<11','average_car_title_balance_dpdlessthan11',
                                           
'0.motortitle_dpd<=0_rate','1.motortitle_0<dpd<10_rate','2.motortitle_10<=dpd<90_rate','3.motortitle_dpd>=90_rate',
                                           'motortt_0.dpd<11','average_motor_title_balance_dpdlessthan11',

'0.normass_dpd<=0_rate','1.normass_0<dpd<10_rate','2.normass_10<=dpd<90_rate','3.normass_dpd>=90_rate','normass_0.dpd<11', 'all_dpd<11_bal',
                                           
'cartt_num_pawn_0. dpd<=0','cartt_num_pawn_1. 0<dpd<10','cartt_num_pawn_2. 10<=dpd<90', 'cartt_num_pawn_3. dpd>=90','cartt_total_pawn',
                                           
'motortt_num_pawn_0. dpd<=0','motortt_num_pawn_1. 0<dpd<10','motortt_num_pawn_2. 10<=dpd<90','motortt_num_pawn_3. dpd>=90','motortt_total_pawn',
                                           
 'motortitle_ontime_rate','cartitle_ontime_rate','motor_title_early_rate_remind','car_title_early_rate_remind',
'diff_clear_reason','diff_no_clear_reason','diff_no_reason','diff_not_found_reason','no_diff','all_cashcount',  
                                           'cashcount_diff_rate','average_cash','median_cash',

'cartt_cancelled','cartt_success_convert','cartt_taking_care','cartt_all',
                                         
                                           'cartt_success_rate','cartt_cancelled_rate',
'motortt_cancelled','motortt_success_convert','motortt_taking_care','motortt_all',
'motortt_success_rate','motortt_cancelled_rate',
                                           
'normass_cancelled','normass_success_convert','normass_taking_care',
'normass_all','normass_success_rate','normass_cancelled_rate',

'total_success','total_cancelled','total_to_shop',
'rate_total_success','rate_total_cancelled',

'cartt_traffic',
'cartt_contract_open',
'cartt_convert_rate',
'motortt_traffic',
'motortt_contract_open',
'motortt_convert_rate',
'normass_traffic',
'normass_contract_open',
'normass_convert_rate',
 'all_traffic',
'all_traffic_contract_open',
'all_traffic_convert_rate',
'nums_diff_day_from_last_direct_audit',
'nums_diff_day_from_last_indirect_audit',
'motor_title_25%',
'car_title_25%',
'normal_asset_25%',
'avg_disbur_pawn_car_title',
'avg_disbur_pawn_motor_title',
'avg_disbur_pawn_normal_asset',
'car_title_nums_day_disbur_rate',
'motor_title_nums_day_disbur_rate',
'normal_asset_nums_day_disbur_rate']]

In [1274]:
template_table_data.columns = ['thang','extract_date','shopcode', 'shopname',  'province', 'latitude', 'longtitude', 'province_id',
                               'province_risk_level',
                              'distance_shop_to_ho_km', 'faults_high_risk', 'faults_low_risk', 'faults_medium_risk', 'faults_significant_risk',
'rolledb0_rate_motor_title', 'rolledb0_rate_car_title',
                               
'growth_rate_car_title', 'growth_rate_motor_title',
       'growth_rate_normal_asset', 'nums_grownet_car_title',
       'nums_grownet_motor_title', 'nums_grownet_normal_asset',
                                           
   'balance_car_title_dpd0_rate', 'balance_car_title_dpd0110_rate',
       'balance_car_title_dpd1190_rate', 'balance_car_title_dpd91_rate',
       'balance_car_title_dpd_lessthan11','average_car_title_balance_dpdlessthan11',
                                           
 'balance_motor_title_dpd0_rate','balance_motor_title_dpd0110_rate', 'balance_motor_title_dpd1190_rate',
       'balance_motor_title_dpd91_rate', 'balance_motor_title_dpd_lessthan11','average_motor_title_balance_dpdlessthan11',

  'balance_norm_ass_dpd0_rate', 'balance_norm_ass_dpd0110_rate',
       'balance_norm_ass_dpd1190_rate', 'balance_norm_ass_dpd91_rate',
       'balance_norm_ass_dpd_lessthan11', 'balance_all_dpd_lessthan11', 
                                           
'num_pawn_car_title_dpd0',  'num_pawn_car_title_dpd0110', 'num_pawn_car_title_dpd1190', 'num_pawn_car_title_dpd91', 'total_pawn_car_title',
                                           
'num_pawn_motor_title_dpd0', 'num_pawn_motor_title_dpd0110', 'num_pawn_motor_title_dpd1190', 'num_pawn_motor_title_dpd91', 'total_pawn_motor_title', 
                                           
 'motor_title_ontime_rate', 'car_title_ontime_rate', 'motor_title_early_rate_remind','car_title_early_rate_remind',
                               
'cash_count_diff_clear_reason', 'cash_count_diff_no_clear_reason', 'cash_count_diff_no_reason', 'cash_count_diff_not_found_reason',
                               'cash_count_no_diff', 'all_cash_count_times', 'cash_count_diff_rate', 'average_cash','median_cash',

'pol_cartt_cancelled','pol_cartt_success_convert','pol_cartt_taking_care','pol_cartt_all',
                                           
                                           'pol_cartt_success_rate','pol_cartt_cancelled_rate',
'pol_motortt_cancelled','pol_motortt_success_convert','pol_motortt_taking_care','pol_motortt_all',
'pol_motortt_success_rate','pol_motortt_cancelled_rate',
                                           
'pol_normass_cancelled','pol_normass_success_convert','pol_normass_taking_care',
'pol_normass_all','pol_normass_success_rate','pol_normass_cancelled_rate',

'pol_total_success','pol_total_cancelled','pol_total_to_shop',
'pol_rate_total_success','pol_rate_total_cancelled',

 'traffic_car_title', 'traffic_car_title_contract_open',

'traffic_car_title_convert_rate', 'traffic_motor_title', 'traffic_motor_title_contract_open',
'traffic_motor_title_convert_rate', 'traffic_normal_asset', 'traffic_normal_asset_contract_open',
'traffic_normal_asset_convert_rate',
                              'all_traffic',
'all_traffic_contract_open',
'all_traffic_convert_rate',
'nums_diff_day_from_last_direct_audit',
'nums_diff_day_from_last_indirect_audit',
'opening_period_motor_title_25%',
'opening_period_car_title_25%',
'opening_period_normal_asset_25%',
'avg_number_disbur_pawn_car_title',
'avg_number_disbur_pawn_motor_title',
'avg_number_disbur_pawn_normal_asset',
'car_title_nums_day_disbur_rate',
'motor_title_nums_day_disbur_rate',
'normal_asset_nums_day_disbur_rate']

In [1275]:
template_table_data = template_table_data.merge(kpi, on =['extract_date','shopcode'], how = 'left' )

In [1276]:
template_table_data = template_table_data.merge(fav, on = ['extract_date','shopname'], how = 'left')

In [1277]:
template_table_data['shopcode'].count()

240

In [1278]:
template_table_data = template_table_data.merge(yiel_index_motor_title,on =['thang','shopcode'], how = 'left' )

In [1279]:
template_table_data = template_table_data.merge(yiel_index_car_title, on =['thang','shopcode'], how = 'left' )

In [1280]:
template_table_data = template_table_data.merge(liquid_rate, on =['thang','shopcode'], how = 'left' )

In [1281]:
template_table_data = template_table_data.merge(closed_pawn_car_title, on = ['extract_date', 'shopcode'], how = 'left')

In [1282]:
template_table_data = template_table_data.merge(closed_pawn_motor_title, on = ['extract_date', 'shopcode'], how = 'left')

In [1283]:
template_table_data = template_table_data.merge(closed_pawn_normal_asset, on = ['extract_date', 'shopcode'], how = 'left')

In [1284]:
template_table_data = template_table_data.merge(shop_qlkv_region, left_on = 'shopcode', right_on = 'Mã PGD', how = 'left')

In [1285]:
template_table_data.columns

Index(['thang', 'extract_date', 'shopcode', 'shopname', 'province', 'latitude',
       'longtitude', 'province_id', 'province_risk_level',
       'distance_shop_to_ho_km',
       ...
       'QLKV', 'Trade Marketing', 'Trade Leader', 'Miền', 'Tỉnh/ Thành phố',
       'Sự cố 14.6', 'Sự cố 18.5', 'Tháng KT', 'Telesales', 'Trạng thái'],
      dtype='object', length=141)

In [1286]:
template_table_data['region_id'] = np.where(template_table_data['Miền'] == 'Miền Bắc', 1, np.where(template_table_data['Miền'] == 'Miền Trung', 2, 3))

In [1287]:
template_table_data['yearmonth'] = [i.strftime('%Y%m') for i in template_table_data['extract_date']]

In [1288]:
template_table_data = template_table_data.merge(asm_risk_lv, left_on = ['yearmonth', 'QLKV'], right_on = ['yearmonth', 'asm_name'], how = 'left')

In [1289]:
template_table_data = template_table_data.merge(shop_age_month, on =['thang','shopcode'], how = 'left' )

In [1290]:
# template_table_data10 = template_table_data[template_table_data['balance_all_dpd_lessthan11'] != 0]

In [1291]:
# template_table_data = template_table_data[template_table_data['staff_tenure']>=0]

In [1292]:
template_table_data['shopcode'].count()

240

SHOP AGE GROUP

In [1293]:
template_table = template_table_data

In [1294]:
compliance_table = template_table[['extract_date',
'shopcode',
'shopname',
'asm_tenure',
'manager_tenure',
'staff_tenure',
'shop_age_m',
'province',
'province_id',
'province_risk_level',
'distance_shop_to_ho_km',
'faults_high_risk',
'faults_low_risk',
'faults_medium_risk',
'faults_significant_risk',
'rolledb0_rate_motor_title',
'growth_rate_car_title',
'growth_rate_motor_title',
'growth_rate_normal_asset',
'nums_grownet_car_title',
'nums_grownet_motor_title',
'nums_grownet_normal_asset',
'balance_car_title_dpd_lessthan11',
'average_car_title_balance_dpdlessthan11',
'balance_motor_title_dpd0_rate',
'balance_motor_title_dpd0110_rate',
'balance_motor_title_dpd_lessthan11',
'average_motor_title_balance_dpdlessthan11',
'balance_norm_ass_dpd0_rate',
'balance_norm_ass_dpd0110_rate',
'balance_norm_ass_dpd_lessthan11',
'balance_all_dpd_lessthan11',
'total_pawn_car_title',                          
'num_pawn_motor_title_dpd0110',
'num_pawn_motor_title_dpd1190',
'total_pawn_motor_title',
'motor_title_ontime_rate',
'car_title_ontime_rate',
'motor_title_early_rate_remind',
'car_title_early_rate_remind',
'fav_rate',
'cash_count_diff_clear_reason',
'cash_count_diff_no_clear_reason',
'cash_count_diff_no_reason',
'cash_count_diff_not_found_reason',
'cash_count_no_diff',
'all_cash_count_times',
'cash_count_diff_rate',
'average_cash',
'median_cash',
'pol_cartt_all',
'pol_cartt_success_rate',
'pol_cartt_cancelled_rate',
'pol_motortt_all',
'pol_motortt_success_rate',
'pol_motortt_cancelled_rate',
'pol_normass_all',
'pol_normass_success_rate',
'pol_normass_cancelled_rate',
'pol_total_to_shop',
'pol_rate_total_success',
'pol_rate_total_cancelled',
'traffic_car_title',
'traffic_car_title_convert_rate',
'traffic_motor_title',
'traffic_motor_title_convert_rate',
'all_traffic',
'all_traffic_contract_open',
'all_traffic_convert_rate',
'kpi_complete_rate',
'nums_diff_day_from_last_direct_audit',
'nums_diff_day_from_last_indirect_audit',
'opening_period_motor_title_25%',
'opening_period_car_title_25%',
'opening_period_normal_asset_25%',
'avg_number_disbur_pawn_car_title',
'avg_number_disbur_pawn_motor_title',
'avg_number_disbur_pawn_normal_asset',
'car_title_nums_day_disbur_rate',
'motor_title_nums_day_disbur_rate',
'normal_asset_nums_day_disbur_rate',
'car_title_closed_rate',
'car_title_early_closed_rate',
'car_title_closed_to_open_new_rate',
'motor_title_closed_rate',
'motor_title_early_closed_rate',
'motor_title_closed_to_open_new_rate',
'normal_asset_closed_rate',
'normal_asset_early_closed_rate',
'normal_asset_closed_to_open_new_rate',
'region_id','asm_name',
'risk_level']]

In [1295]:
compliance_table = compliance_table.rename(columns={'province':'province_name'})

In [1296]:
compliance_table['shop_age_group'] = np.where(compliance_table['shop_age_m']<4, 'running_less_than_3m', np.where((compliance_table['shop_age_m']>3) & (compliance_table['shop_age_m']<7), 'running_4-6m',np.where((compliance_table['shop_age_m']>6) & (compliance_table['shop_age_m']<13),'running_7-12m', np.where((compliance_table['shop_age_m']>12) & (compliance_table['shop_age_m']<25),'running_13-24m', 'running_more_than_24m'))))

In [1297]:
compliance_table = compliance_table.rename(columns={'risk_level':'asm_risk_level'})

In [1298]:
compliance_table.head(5)

,extract_date,shopcode,shopname,asm_tenure,manager_tenure,staff_tenure,shop_age_m,province_name,province_id,province_risk_level,distance_shop_to_ho_km,faults_high_risk,faults_low_risk,faults_medium_risk,faults_significant_risk,rolledb0_rate_motor_title,growth_rate_car_title,growth_rate_motor_title,growth_rate_normal_asset,nums_grownet_car_title,nums_grownet_motor_title,nums_grownet_normal_asset,balance_car_title_dpd_lessthan11,average_car_title_balance_dpdlessthan11,balance_motor_title_dpd0_rate,balance_motor_title_dpd0110_rate,balance_motor_title_dpd_lessthan11,average_motor_title_balance_dpdlessthan11,balance_norm_ass_dpd0_rate,balance_norm_ass_dpd0110_rate,balance_norm_ass_dpd_lessthan11,balance_all_dpd_lessthan11,total_pawn_car_title,num_pawn_motor_title_dpd0110,num_pawn_motor_title_dpd1190,total_pawn_motor_title,motor_title_ontime_rate,car_title_ontime_rate,motor_title_early_rate_remind,car_title_early_rate_remind,fav_rate,cash_count_diff_clear_reason,cash_count_diff_no_clear_reason,cash_count_diff_no_reason,cash_count_diff_not_found_reason,cash_count_no_diff,all_cash_count_times,cash_count_diff_rate,average_cash,median_cash,pol_cartt_all,pol_cartt_success_rate,pol_cartt_cancelled_rate,pol_motortt_all,pol_motortt_success_rate,pol_motortt_cancelled_rate,pol_normass_all,pol_normass_success_rate,pol_normass_cancelled_rate,pol_total_to_shop,pol_rate_total_success,pol_rate_total_cancelled,traffic_car_title,traffic_car_title_convert_rate,traffic_motor_title,traffic_motor_title_convert_rate,all_traffic,all_traffic_contract_open,all_traffic_convert_rate,kpi_complete_rate,nums_diff_day_from_last_direct_audit,nums_diff_day_from_last_indirect_audit,opening_period_motor_title_25%,opening_period_car_title_25%,opening_period_normal_asset_25%,avg_number_disbur_pawn_car_title,avg_number_disbur_pawn_motor_title,avg_number_disbur_pawn_normal_asset,car_title_nums_day_disbur_rate,motor_title_nums_day_disbur_rate,normal_asset_nums_day_disbur_rate,car_title_closed_rate,car_title_early_closed_rate,car_title_closed_to_open_new_rate,motor_title_closed_rate,motor_title_early_closed_rate,motor_title_closed_to_open_new_rate,normal_asset_closed_rate,normal_asset_early_closed_rate,normal_asset_closed_to_open_new_rate,region_id,asm_name,asm_risk_level,shop_age_group
0,2020-11-10,BDG0227,BDG0227.Tân Phước Khánh,NaN,13.0,5.0,3.0,Bình Dương,61,5,27.161646,0.0,0.0,0.0,0.0,0.046820,0.000000,0.203764,0.416667,1,3,1,2.000000e+07,2.000000e+07,0.962948,0.037052,1.187391e+09,9.813148e+06,0.558824,0.441176,6866028.0,1.214257e+09,1.0,4.0,0.0,121.0,0.600000,0.000000,1.000000,0.00,NaN,3.0,0.0,0.0,0.0,7.0,10.0,0.002439,23312000.0,20923250,0.0,0.0,0.0,51,0.137255,0.411765,144,0.027778,0.520833,195.0,0.056410,0.492308,0.0,0.0,27,1.000000,30.0,30.0,1.000000,0.821389,74,74,36.5,NaN,6.00,NaN,3.000000,1.0,0.0,0.9,0.3,NaN,NaN,NaN,0.030534,0.000000,0.5,0.400000,0.400000,0.0,3,Hồ Xuân Dũng,3,running_less_than_3m
1,2020-11-10,BLU0159,BLU0159.43 Trần Phú,NaN,20.0,22.0,8.0,Bạc Liêu,94,4,193.848178,0.0,1.0,0.0,0.0,0.024170,0.000000,0.057647,-0.231442,3,3,2,8.515149e+08,1.064394e+08,0.862707,0.019685,4.384968e+09,7.369694e+06,1.000000,0.000000,93219524.0,5.329702e+09,8.0,15.0,68.0,665.0,0.787500,0.750000,0.950000,0.75,NaN,5.0,2.0,0.0,0.0,3.0,10.0,0.022475,49372500.0,51365000,0.0,0.0,0.0,48,0.104167,0.458333,224,0.013393,0.531250,272.0,0.029412,0.518382,0.0,0.0,71,0.788732,73.0,58.0,0.794521,0.973365,194,102,26.0,NaN,4.25,NaN,5.600000,1.0,0.0,1.0,0.2,NaN,NaN,NaN,0.017266,0.002878,0.0,0.208333,0.041667,0.0,3,Nguyễn Đình Long Vinh,3,running_7-12m
2,2020-11-10,BNH0146,BNH0146.147 Nguyễn Văn Cừ,NaN,20.0,14.0,9.0,Bắc Ninh,99,3,32.582802,0.0,0.0,0.0,0.0,0.029420,-0.088653,-0.023064,-0.231783,3,3,2,1.541991e+09,1.284993e+08,0.878391,0.024874,2.050522e+09,9.113431e+06,0.905522,0.094478,180964090.0,3.773477e+09,13.0,5.0,23.0,250.0,0.625000,0.333333,1.000000,1.00,NaN,6.0,0.0,0.0,3.0,1.0,10.0,0.019782,75449000.0,74016000,0.0,0.0,0.0,12,0.250000,0.333333,23,0.000000,0.434783,35.0,0.085714,0.40

TRAIN AND PREDICT

In [920]:
train_data = pd.read_excel(r'D:\F88\Python code\shop_score_source\official_code_for_shop_warning\T10\ngay_30.xlsx', sheet_name='training')

In [1299]:
test_data = compliance_table

In [1300]:
test_data['day'] = [i.day for i in test_data['extract_date']]
# [datetime.strptime(i, "%Y-%m-%d").day for i in test_data['extract_date']]

In [1105]:
sub_train_dataset = train_data[['day',
'shop_age_group',
'extract_date',
'shopname',
'Compliance_risk_Rating',
'shop_age_m',
'province_name',
'province_id',
'province_risk_level',
'asm_name',
'asm_risk_level',
'distance_shop_to_ho_km',
'faults_high_risk',
'faults_low_risk',
'faults_medium_risk',
'faults_significant_risk',
'rolledb0_rate_motor_title',
'growth_rate_car_title',
'growth_rate_motor_title',
'growth_rate_normal_asset',
'nums_grownet_car_title',
'nums_grownet_motor_title',
'nums_grownet_normal_asset',
'balance_motor_title_dpd0_rate',
'balance_motor_title_dpd0110_rate',
'balance_norm_ass_dpd0_rate',
'balance_norm_ass_dpd0110_rate',
'balance_all_dpd_lessthan11',
'total_pawn_car_title',
'total_pawn_motor_title',
'motor_title_ontime_rate',
'car_title_ontime_rate',
'motor_title_early_rate_remind',
'car_title_early_rate_remind',
'fav_rate',
'cash_count_diff_no_clear_reason',
'cash_count_diff_no_reason',
'cash_count_diff_not_found_reason',
'all_cash_count_times',
'cash_count_diff_rate',
'median_cash',
'pol_cartt_all',
'pol_cartt_success_rate',
'pol_total_to_shop',
'pol_rate_total_success',
'traffic_car_title',
'traffic_car_title_convert_rate',
'all_traffic',
'all_traffic_convert_rate',
'kpi_complete_rate',
'nums_diff_day_from_last_direct_audit',
'nums_diff_day_from_last_indirect_audit',
'opening_period_motor_title_25%',
'opening_period_normal_asset_25%',
'avg_number_disbur_pawn_car_title',
'avg_number_disbur_pawn_motor_title',
'avg_number_disbur_pawn_normal_asset',
'car_title_nums_day_disbur_rate',
'motor_title_nums_day_disbur_rate',
'normal_asset_nums_day_disbur_rate',
'car_title_closed_rate',
'motor_title_closed_rate',
'normal_asset_closed_rate',
'region_id']]

In [1301]:
# 'Compliance_risk_Rating',
sub_test_dataset = test_data[['day',
'shop_age_group',
'extract_date',
'shopname',

'shop_age_m',
'province_name',
'province_id',
'province_risk_level',
'asm_name',
'asm_risk_level',
'distance_shop_to_ho_km',
'faults_high_risk',
'faults_low_risk',
'faults_medium_risk',
'faults_significant_risk',
'rolledb0_rate_motor_title',
'growth_rate_car_title',
'growth_rate_motor_title',
'growth_rate_normal_asset',
'nums_grownet_car_title',
'nums_grownet_motor_title',
'nums_grownet_normal_asset',
'balance_motor_title_dpd0_rate',
'balance_motor_title_dpd0110_rate',
'balance_norm_ass_dpd0_rate',
'balance_norm_ass_dpd0110_rate',
'balance_all_dpd_lessthan11',
'total_pawn_car_title',
'total_pawn_motor_title',
'motor_title_ontime_rate',
'car_title_ontime_rate',
'motor_title_early_rate_remind',
'car_title_early_rate_remind',
'fav_rate',
'cash_count_diff_no_clear_reason',
'cash_count_diff_no_reason',
'cash_count_diff_not_found_reason',
'all_cash_count_times',
'cash_count_diff_rate',
'median_cash',
'pol_cartt_all',
'pol_cartt_success_rate',
'pol_total_to_shop',
'pol_rate_total_success',
'traffic_car_title',
'traffic_car_title_convert_rate',
'all_traffic',
'all_traffic_convert_rate',
'kpi_complete_rate',
'nums_diff_day_from_last_direct_audit',
'nums_diff_day_from_last_indirect_audit',
'opening_period_motor_title_25%',
'opening_period_normal_asset_25%',
'avg_number_disbur_pawn_car_title',
'avg_number_disbur_pawn_motor_title',
'avg_number_disbur_pawn_normal_asset',
'car_title_nums_day_disbur_rate',
'motor_title_nums_day_disbur_rate',
'normal_asset_nums_day_disbur_rate',
'car_title_closed_rate',
'motor_title_closed_rate',
'normal_asset_closed_rate',
'region_id']]

In [1113]:
x_train_dataset = sub_train_dataset.drop(columns=['shop_age_group',
'extract_date',
'shopname',
'Compliance_risk_Rating',
'province_name',
'province_id',
'asm_name'])

y_train_dataset = sub_train_dataset[['Compliance_risk_Rating']]

In [1302]:
# 'Compliance_risk_Rating',
x_test_dataset = sub_test_dataset.drop(columns=['shop_age_group',
'extract_date',
'shopname',

'province_name',
'province_id',
'asm_name'])

In [1303]:
x_train_dataset = x_train_dataset.replace(r'^\s*$',0, regex=True)
x_train_dataset = x_train_dataset.replace([np.inf, -np.inf],0)
x_train_dataset = x_train_dataset.fillna(0)

In [1304]:
x_test_dataset = x_test_dataset.replace(r'^\s*$',0, regex=True)
x_test_dataset = x_test_dataset.replace([np.inf, -np.inf],0)
x_test_dataset = x_test_dataset.fillna(0)

In [1119]:
x_train_dataset = sm.add_constant(x_train_dataset)

In [1306]:
x_test_dataset['const'] = 1.0

In [1307]:
x_test_dataset = x_test_dataset[['const', 'day', 'shop_age_m', 'province_risk_level', 'asm_risk_level',
       'distance_shop_to_ho_km', 'faults_high_risk', 'faults_low_risk',
       'faults_medium_risk', 'faults_significant_risk',
       'rolledb0_rate_motor_title', 'growth_rate_car_title',
       'growth_rate_motor_title', 'growth_rate_normal_asset',
       'nums_grownet_car_title', 'nums_grownet_motor_title',
       'nums_grownet_normal_asset', 'balance_motor_title_dpd0_rate',
       'balance_motor_title_dpd0110_rate', 'balance_norm_ass_dpd0_rate',
       'balance_norm_ass_dpd0110_rate', 'balance_all_dpd_lessthan11',
       'total_pawn_car_title', 'total_pawn_motor_title',
       'motor_title_ontime_rate', 'car_title_ontime_rate',
       'motor_title_early_rate_remind', 'car_title_early_rate_remind',
       'fav_rate', 'cash_count_diff_no_clear_reason',
       'cash_count_diff_no_reason', 'cash_count_diff_not_found_reason',
       'all_cash_count_times', 'cash_count_diff_rate', 'median_cash',
       'pol_cartt_all', 'pol_cartt_success_rate', 'pol_total_to_shop',
       'pol_rate_total_success', 'traffic_car_title',
       'traffic_car_title_convert_rate', 'all_traffic',
       'all_traffic_convert_rate', 'kpi_complete_rate',
       'nums_diff_day_from_last_direct_audit',
       'nums_diff_day_from_last_indirect_audit',
       'opening_period_motor_title_25%', 'opening_period_normal_asset_25%',
       'avg_number_disbur_pawn_car_title',
       'avg_number_disbur_pawn_motor_title',
       'avg_number_disbur_pawn_normal_asset', 'car_title_nums_day_disbur_rate',
       'motor_title_nums_day_disbur_rate', 'normal_asset_nums_day_disbur_rate',
       'car_title_closed_rate', 'motor_title_closed_rate',
       'normal_asset_closed_rate', 'region_id']]

In [1308]:
ols_model = sm.OLS(y_train_dataset,x_train_dataset)

ols_result = ols_model.fit()

In [1309]:
x_test_dataset['province_risk_level'] = [int(i) for i in x_test_dataset['province_risk_level']]
x_test_dataset['asm_risk_level'] = [int(i) for i in x_test_dataset['asm_risk_level']]

In [1132]:
# x_test_dataset = pd.read_excel('x_test_dataset.xlsx')

In [1310]:
sub_test_dataset['ols_pred_score'] = ols_result.predict(x_test_dataset)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1311]:
sub_test_dataset['ols_pred_score'] = sub_test_dataset['ols_pred_score']*100

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1312]:
all_final = sub_test_dataset[['day',
'shop_age_group',
'extract_date',
'shopname',
'ols_pred_score',              
'shop_age_m',
'province_name',
'province_id',
'province_risk_level',
'asm_name',
'asm_risk_level',
'distance_shop_to_ho_km',
'faults_high_risk',
'faults_low_risk',
'faults_medium_risk',
'faults_significant_risk',
'rolledb0_rate_motor_title',
'growth_rate_car_title',
'growth_rate_motor_title',
'growth_rate_normal_asset',
'nums_grownet_car_title',
'nums_grownet_motor_title',
'nums_grownet_normal_asset',
'balance_motor_title_dpd0_rate',
'balance_motor_title_dpd0110_rate',
'balance_norm_ass_dpd0_rate',
'balance_norm_ass_dpd0110_rate',
'balance_all_dpd_lessthan11',
'total_pawn_car_title',
'total_pawn_motor_title',
'motor_title_ontime_rate',
'car_title_ontime_rate',
'motor_title_early_rate_remind',
'car_title_early_rate_remind',
'fav_rate',
'cash_count_diff_no_clear_reason',
'cash_count_diff_no_reason',
'cash_count_diff_not_found_reason',
'all_cash_count_times',
'cash_count_diff_rate',
'median_cash',
'pol_cartt_all',
'pol_cartt_success_rate',
'pol_total_to_shop',
'pol_rate_total_success',
'traffic_car_title',
'traffic_car_title_convert_rate',
'all_traffic',
'all_traffic_convert_rate',
'kpi_complete_rate',
'nums_diff_day_from_last_direct_audit',
'nums_diff_day_from_last_indirect_audit',
'opening_period_motor_title_25%',
'opening_period_normal_asset_25%',
'avg_number_disbur_pawn_car_title',
'avg_number_disbur_pawn_motor_title',
'avg_number_disbur_pawn_normal_asset',
'car_title_nums_day_disbur_rate',
'motor_title_nums_day_disbur_rate',
'normal_asset_nums_day_disbur_rate',
'car_title_closed_rate',
'motor_title_closed_rate',
'normal_asset_closed_rate',
'region_id']]

In [1313]:
all_final['Compliance_risk_Rating'] = np.nan

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1314]:
all_final = all_final[['day', 'shop_age_group', 'extract_date', 'shopname',
       'Compliance_risk_Rating', 'ols_pred_score', 'shop_age_m',
       'province_name', 'province_id', 'province_risk_level', 'asm_name',
       'asm_risk_level', 'distance_shop_to_ho_km', 'faults_high_risk',
       'faults_low_risk', 'faults_medium_risk', 'faults_significant_risk',
       'rolledb0_rate_motor_title', 'growth_rate_car_title',
       'growth_rate_motor_title', 'growth_rate_normal_asset',
       'nums_grownet_car_title', 'nums_grownet_motor_title',
       'nums_grownet_normal_asset', 'balance_motor_title_dpd0_rate',
       'balance_motor_title_dpd0110_rate', 'balance_norm_ass_dpd0_rate',
       'balance_norm_ass_dpd0110_rate', 'balance_all_dpd_lessthan11',
       'total_pawn_car_title', 'total_pawn_motor_title',
       'motor_title_ontime_rate', 'car_title_ontime_rate',
       'motor_title_early_rate_remind', 'car_title_early_rate_remind',
       'fav_rate', 'cash_count_diff_no_clear_reason',
       'cash_count_diff_no_reason', 'cash_count_diff_not_found_reason',
       'all_cash_count_times', 'cash_count_diff_rate', 'median_cash',
       'pol_cartt_all', 'pol_cartt_success_rate', 'pol_total_to_shop',
       'pol_rate_total_success', 'traffic_car_title',
       'traffic_car_title_convert_rate', 'all_traffic',
       'all_traffic_convert_rate', 'kpi_complete_rate',
       'nums_diff_day_from_last_direct_audit',
       'nums_diff_day_from_last_indirect_audit',
       'opening_period_motor_title_25%', 'opening_period_normal_asset_25%',
       'avg_number_disbur_pawn_car_title',
       'avg_number_disbur_pawn_motor_title',
       'avg_number_disbur_pawn_normal_asset', 'car_title_nums_day_disbur_rate',
       'motor_title_nums_day_disbur_rate', 'normal_asset_nums_day_disbur_rate',
       'car_title_closed_rate', 'motor_title_closed_rate',
       'normal_asset_closed_rate', 'region_id']]

In [1315]:
update_table = all_final[all_final['shop_age_group']!='running_less_than_3m']

In [1316]:
update_data(control_wb,'10/20/25/30-Compliance',update_table)

In [1137]:
# all_final.to_excel('sub_test_dataset.xlsx', index = False)

In [9]:
# all_final = pd.read_excel('sub_test_dataset.xlsx')

In [2]:
# import pygsheets

In [31]:
# client = pygsheets.authorize(service_account_file=r'D:\F88\Python code\Immediate_warning\service_account.json')

# temp_wb_id = '1-XjnJNeEwEuk13_xQjmewvZ6aAXFRXSlcEcQpaOq3vE'
# # 1yapjrHmzWCEUr_CL94aUiNo7TLMKLPTyMefNiDZ8MmE
# sheet = client.open_by_key(temp_wb_id)



# # raw_sheet = sheet.worksheet_by_title('raw_data_new')
# raw_sheet = sheet.worksheet_by_title('20/25/30-T10')
# # raw_sheet_1 = sheet.worksheet_by_title('running-shop-1-3m')
# # raw_sheet_2 = sheet.worksheet_by_title('running-shop-4-6m')
# # raw_sheet_3 = sheet.worksheet_by_title('running-shop-7-12m')
# # raw_sheet_4 = sheet.worksheet_by_title('running-shop-13-24m')
# # raw_sheet_5 = sheet.worksheet_by_title('running-shop-more-than-24m')

In [32]:
# raw_sheet.set_dataframe(template_data, (2,1))

D:\Anaconda3\lib\site-packages\pygsheets\worksheet.py:1304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype('unicode').replace('<NA>', nan)
